In [4]:
import pandas as pd, numpy as np

urls = {
    'weapons':'https://dnd5e.wikidot.com/weapons',
    'armor':'https://dnd5e.wikidot.com/armor',
    'character lineage':'https://dnd5e.wikidot.com/lineage',
}

## Pulling Character Lineage Information

In [5]:
df = pd.read_html('https://dnd5e.wikidot.com/lineage')

In [2]:
from bs4 import BeautifulSoup as BS
import requests,re

resp = requests.get('https://dnd5e.wikidot.com/lineage')
baseURL = 'https://dnd5e.wikidot.com'
bs = BS(resp.content)

In [ ]:
#get limited set of character races and put HTML elsewhere for parsing
for i in range(0,4):
    f = df[i].reset_index().melt(id_vars='index')
    for val in f['value']:
        # print(val)
        if type(val) == str:
            l = [e.parent for e in bs(text=re.compile(r'^'+val+'$'))]
            if len(l) > 0:
                resp = requests.get(url=baseURL+l[0]['href'])
                soup = BS(resp.content)
                html = soup.prettify()
                # print('./data/'+val+'.html')
                with open('./data/lineage/'+val+'.html','w') as file:
                    file.write(html)
                    file.close()
              


## Pulling Character Background Information

In [ ]:
#extract content for all character backgrounds for further parsing
resp = requests.get(baseURL)
bs = BS(resp.content)
links = bs(href=re.compile(r'/background:.*'))
for l in links:
    resp = requests.get(baseURL+l['href'])
    soup = BS(resp.content)   
    with open('./data/backgrounds/'+l.text+'.html','w') as file:
        file.write(html)
        file.close()

## Pulling Spell Information

In [125]:
resp = requests.get(baseURL)
bs = BS(resp.content)
links = bs(href=re.compile(r'/spells:.*'))
char_spell_pages = [
    {
        'link':baseURL+link['href'], 
        'class':link.text.split(' ')[0], 
        'proc':link.text
    } 
    for link in links if "Spells" in link.text
]
print(char_spell_pages)
all_spells = []
spell_links = set()
pass
for pg in char_spell_pages:
    resp = requests.get(pg['link'])
    soup = BS(resp.content)
    tables = soup(id=re.compile(r'wiki-tab-.*'))
    spell_list = []
    for i in range(len(tables)):
        table = tables[i]
        hrefs = table.find_all('a')
        names = [x.text for x in hrefs]
        hrefs = [baseURL+x['href'] for x in hrefs]
        data = [(x,y) for x,y in zip(names,hrefs)]
        spell_links.update(data)
        tab = pd.read_html(table.decode_contents())[0]
        tab['class'] = pg['class']
        if i == 0:
            level = 'Cantrip'
        else:
            level = str(i)
        tab['level'] = level
        print(pg['class']+':'+level)
        spell_list.append(tab)
    all_spells.extend(spell_list)

spell_df = pd.concat(all_spells)
spell_df.to_csv('./data/spellbook.csv',index=False)

[{'link': 'https://dnd5e.wikidot.com/spells:artificer', 'class': 'Artificer', 'proc': 'Artificer Spells'}, {'link': 'https://dnd5e.wikidot.com/spells:bard', 'class': 'Bard', 'proc': 'Bard Spells'}, {'link': 'https://dnd5e.wikidot.com/spells:cleric', 'class': 'Cleric', 'proc': 'Cleric Spells'}, {'link': 'https://dnd5e.wikidot.com/spells:druid', 'class': 'Druid', 'proc': 'Druid Spells'}, {'link': 'https://dnd5e.wikidot.com/spells:paladin', 'class': 'Paladin', 'proc': 'Paladin Spells'}, {'link': 'https://dnd5e.wikidot.com/spells:ranger', 'class': 'Ranger', 'proc': 'Ranger Spells'}, {'link': 'https://dnd5e.wikidot.com/spells:sorcerer', 'class': 'Sorcerer', 'proc': 'Sorcerer Spells'}, {'link': 'https://dnd5e.wikidot.com/spells:warlock', 'class': 'Warlock', 'proc': 'Warlock Spells'}, {'link': 'https://dnd5e.wikidot.com/spells:wizard', 'class': 'Wizard', 'proc': 'Wizard Spells'}]
Artificer:Cantrip
Artificer:1
Artificer:2
Artificer:3
Artificer:4
Artificer:5
Bard:Cantrip
Bard:1
Bard:2
Bard:3
Ba

In [ ]:
for link in spell_links:
    name,url = link[0],link[1]
    resp = requests.get(url)
    soup = BS(resp.content)
    with open('./data/{}.html'.format(re.sub('(:|/)','-',name)),'w') as file:
        file.write(soup.prettify())
        file.close()

In [128]:
broken = []
for link in spell_links:
    if link[0] == '':
        broken.append(link)

print(broken)

[]


In [323]:
len(spell_links)

580

In [318]:
import glob, json
page_bodies = []
not_downloaded = []
files = [(s[0],'./data/spells/{}.html'.format(s[0]),s[1]) for s in spell_links]
for f in files:
    # print(f[1])
    try:
        soup = BS(open(f[1],'r').read())
        body = soup(id=re.compile(r'page-content*'))[0]
        title = soup('title')[0].text
        title = title.split('-')[0].strip()
        # print(title,f[0],f[1])
    except:
        not_downloaded.append((f[0],f[2]))
        pass
        # print('file {} not found.  link at {}'.format(f[0],f[1]))
not_downloaded

[('Wall of Sand', 'https://dnd5e.wikidot.com/spell:wall-of-sand'),
 ('Create Homunculus', 'https://dnd5e.wikidot.com/spell:create-homunculus'),
 ('Immovable Object', 'https://dnd5e.wikidot.com/spell:immovable-object'),
 ('Passwall', 'https://dnd5e.wikidot.com/spell:passwall'),
 ('Lightning Arrow', 'https://dnd5e.wikidot.com/spell:lightning-arrow'),
 ('Arcane Hacking (UA)', 'https://dnd5e.wikidot.com/spell:arcane-hacking'),
 ('Storm Sphere', 'https://dnd5e.wikidot.com/spell:storm-sphere'),
 ('Summon Undead', 'https://dnd5e.wikidot.com/spell:summon-undead'),
 ('Conjure Knowbot (UA)', 'https://dnd5e.wikidot.com/spell:conjure-knowbot'),
 ('Phantom Steed', 'https://dnd5e.wikidot.com/spell:phantom-steed'),
 ('Summon Aberration', 'https://dnd5e.wikidot.com/spell:summon-aberration'),
 ('Telepathy', 'https://dnd5e.wikidot.com/spell:telepathy'),
 ('Hunger Of Hadar', 'https://dnd5e.wikidot.com/spell:hunger-of-hadar'),
 ('Contact Other Plane',
  'https://dnd5e.wikidot.com/spell:contact-other-plane

In [324]:
for link in not_downloaded:
    name,url = link[0],link[1]
    resp = requests.get(url)
    soup = BS(resp.content)
    print(len(resp.content))
    print(re.sub('(:|/)','-',name))
    with open('./data/spells/{}.html'.format(re.sub('(:|/)','-',name)),'w') as file:
        file.write(soup.prettify())
        file.close()

20030
Wall of Sand
20918
Create Homunculus
20894
Immovable Object
20167
Passwall
20508
Lightning Arrow
20867
Arcane Hacking (UA)
20778
Storm Sphere
23957
Summon Undead
21390
Conjure Knowbot (UA)
20399
Phantom Steed
23537
Summon Aberration
20275
Telepathy
20436
Hunger Of Hadar
20868
Contact Other Plane
20550
Create Magen
20372
Id Insinuation (UA)
20318
Finger of Death
20128
Jim's Glowing Coin
20297
Snilloc's Snowball Swarm
21130
Simulacrum
20410
Temporal Shunt
22009
Magic Jar
21225
Conjure Hezrou (UA)
20354
On-Off (UA)
20188
Magic Missile
20020
Mage Armor
20182
Power Word- Kill
21358
Wall of Light
20383
Vitriolic Sphere
20500
Pulse Wave
20644
Witch Bolt
23398
Spirit Of Death
20556
Vampiric Touch
21329
Otiluke's Freezing Sphere
21398
Conjure Vrock (UA)
19963
Misty Step
20103
Conjure Barrage
21721
Summon Greater Demon
20475
Gravity Fissure
20655
Fortune's Favor
20839
Contingency
20188
Conjure Volley
20559
Mind Spike
20886
Disintegrate
21460
Galder's Tower
20030
Sapping Sting
20481
Acid St

## Parsing Spell Information

In [269]:
import glob,json

files = glob.glob('./data/spells/*.html')
page_bodies = []
for f in files:
    soup = BS(open(f,'r').read())
    body = soup(id=re.compile(r'page-content*'))[0]
    title = soup('title')[0].text
    title = title.split('-')[0].strip()
    # print(title)
    #print(body.prettify())
    # page_bodies.append(body)
    description = re.findall('(?<=Duration:).*?(?=\<em\>)',body.decode_contents(),re.DOTALL)
    # description = re.findall('(?<=\</p\>\n\<p\>\n).*?(?=\<strong\>)',description,re.DOTALL)
    # print(description[0].strip())
    higher_level_mod = re.findall('(?<=At Higher Levels\.).*?(?=\</p\>)',body.decode_contents(),re.DOTALL)
    # spell_does_damage = re.findall()
    # saving_throw = re.findall('succeed on a ([a-zA-Z]+) saving throw',body.decode_contents())
    # if len(saving_throw) == 0:
    #     s1 = ''
    # else:
    #     s1=saving_throw[0]
    # saving_throw2 = re.findall('make a ([a-zA-Z]+) saving throw',body.decode_contents())
    # if len(saving_throw2) == 0:
    #     s2 = ''
    # else:
    #     s2=saving_throw2[0]

    saving_throw3 = re.findall('([a-zA-Z]+) saving throw',body.decode_contents())
    if len(saving_throw3) == 0:
        s3 = ''
    else:
        s3=saving_throw3[0]
    print(title)
    has_saving_throw = len(saving_throw3) > 0
    save_throw_impact = re.findall("fail|succ.*?(?= )",body.decode_contents())
    if has_saving_throw:
        print('\tsaving throw: {}'.format(s3))
        print('\ton: {}'.format(save_throw_impact))

    deals_damage = re.findall('(?<=take|deal).*?(?=damage)',body.decode_contents())

    # damage types = piercing bludgeoning slashing cold fire lightning thunder poison acid necrotic force radiant psychic
    if len(deals_damage) > 0:
        dmg_dice_type = re.findall('(\d+d\d+) (piercing|bludgeoning|slashing|cold|fire|lightning|thunder|poison|acid|necrotic|force|radiant|psychic)',body.decode_contents())
        if len(dmg_dice_type) > 0:
            dice_type_qty = dmg_dice_type[0][0].split('d')
            dmg_type = dmg_dice_type[0][1]
            dice_qty = dice_type_qty[0]
            dice_size = dice_type_qty[1]
            print('\tdmg finding: {}\n\tnumber of dice:{}\n\tdice size:{}\n\tdamage type: {}'.format(deals_damage,dice_qty,dice_size,dmg_type))
        # print('\tdeals damage: {}'.format(deals_damage))

    if len(higher_level_mod) > 0:
        # print(higher_level_mod)
        per_level_increase = re.findall(re.compile(r'of \d{1,2}[a-z]{2} level or higher.*?(?=for each)'),higher_level_mod[0])
        per_level_increase2 = []

        at_level_increase = re.findall('(?<=when you reach).*?(?=\n)',higher_level_mod[0])
        at_level_increase2 = []

        if len(at_level_increase) > 0:
            at_level_increase2 = re.findall('(\d{1,2}[a-zA-Z]{2}) level \((\dd\d)\)',at_level_increase[0])
            at_lvl_increase = {}
            for item in at_level_increase2:
                lvl_val = int(item[0][:-2])
                dice_type_qty = item[1].split('d')
                dice_qty = int(dice_type_qty[0])
                dice_size=int(dice_type_qty[1])
                at_lvl_increase[lvl_val]={'dice_qty':dice_qty,'dice_size':dice_size}
            # print('\tat level increases: {}'.format(at_level_increase2))
            print('\tat level increases: {}'.format(json.dumps(at_lvl_increase,indent=2)))
        if len(per_level_increase) > 0:
            min_level = re.findall('(?<=of )(\d+[a-zA-Z]{2})(?= level)',per_level_increase[0])
            inc_by = re.findall('(?<=increase).*',per_level_increase[0])
            if len(inc_by) > 0:
                inc_by = re.findall('(?<=by ).*',per_level_increase[0])
            if len(inc_by) > 0:
                inc_by = inc_by[0]
            if len(min_level) > 0:
                min_level = min_level[0][:-2]
            print('\tper level increases: {}\n\tmin level: {}\n\tincrease by: {}'.format(per_level_increase,min_level,inc_by))
    if len(description) > 0:
        desc = list.copy(description)
        desc = desc[0]
        desc = desc[desc.index('<p>'):]
        p = re.compile(r'<.*?>')
        desc = re.sub(p,' ',desc)
        # desc = desc[desc.index('\n'):]
        desc = re.sub(r'\s+', ' ', desc)
        
        print('\tdesc:{}'.format(desc))
        print('\tdescription: {}'.format(description))
        # print('\tdescription: {}'.format(re.sub('\<.*>','',description[0].strip())))
    #succeed on a Dexterity saving throw

Absorb Elements
	per level increases: ['of 2nd level or higher, the extra damage increases by 1d6 ']
	min level: 2
	increase by: 1d6 
	desc: The spell captures some of the incoming energy, lessening its effect on you and storing it for your next melee attack. You have resistance to the triggering damage type until the start of your next turn. Also, the first time you hit with a melee attack on your next turn, the target takes an extra 1d6 damage of the triggering type, and the spell ends. 
	description: ['\n            </strong>\n            1 round\n           </p>\n<p>\n            The spell captures some of the incoming energy, lessening its effect on you and storing it for your next melee attack. You have resistance to the triggering damage type until the start of your next turn. Also, the first time you hit with a melee attack on your next turn, the target takes an extra 1d6 damage of the triggering type, and the spell ends.\n           </p>\n<p>\n<strong>\n']
Acid Splash
	saving 

In [194]:
st = '\n            </strong>\n            Instantaneous\n           </p>\n<p>\n            You hurl a bubble of acid. Choose one creature you can see within range, or choose two creatures you can see within range that are within 5 feet of each other. A target must succeed on a Dexterity saving throw or take 1d6 acid damage.\n           </p>\n<p>\n<strong>\n'

In [270]:
re.findall('(?<=take).*?(?=damage)',st)

[' 1d6 acid ']

req_atk_roll - T/F
req_save_throw - T/F
  on_save_success - 
    dmg_dice_qty
    dmg_dice_size
  on_save_fail - 
    dmg_dice_q
deals_dmg T/F
dmg_dice_qty - number
dmg_dice_size - number
has_at_level_increase T/F
at_level_increase list of dictionaries {'level':{'dice_qty':int,'dice_size':int}}
has_per_level_increase T/F

  min_level_for_increase - int
  addtl_dice_per_level - int


In [325]:
import glob,json
require_saves = []
files = glob.glob('./data/spells/*.html')
page_bodies = []
for f in files:
    soup = BS(open(f,'r').read())
    body = soup(id=re.compile(r'page-content*'))[0]
    title = soup('title')[0].text
    title = title.split('-')[0].strip()
    description = re.findall('(?<=Duration:).*?(?=\<em\>)',body.decode_contents(),re.DOTALL)
    saving_throw3 = re.findall('([a-zA-Z]+) saving throw',body.decode_contents())
    if len(saving_throw3) == 0:
        s3 = ''
    else:
        s3=saving_throw3[0]
    # print(title)
    if len(description) > 0:
        desc = list.copy(description)
        desc = desc[0]
        desc = desc[desc.index('<p>'):]
        p = re.compile(r'<.*?>')
        desc = re.sub(p,' ',desc)
        # desc = desc[desc.index('\n'):]
        desc = re.sub(r'\s+', ' ', desc)
    has_saving_throw = len(saving_throw3) > 0
    save_throw_impact = re.findall("fail|succ.*?(?= )",body.decode_contents())
    if has_saving_throw:
        require_saves.append({
            'spell':title,
            'saving_throw':s3,
            'desc':desc
        })
        # print(title)
        # print('\tsaving throw: {}'.format(s3))
        # print('\ton: {}'.format(save_throw_impact))
        # print('\tdesc:{}'.format(desc))
        # print('\tdescription: {}'.format(description))
        # print('\tdescription: {}'.format(re.sub('\<.*>','',description[0].strip())))
    #succeed on a Dexterity saving throw

In [298]:
fail_saves = []
succeed_saves = []
other_saves = []
for d in require_saves:
    r1 = re.findall('failed save',d['desc'])
    if len(r1) > 0:
        fail_saves.append(d)
    r2 = re.findall('succeed on a|successful.*?(?=saving throw)',d['desc'])
    if len(r2) > 0:
        succeed_saves.append(d)
    if len(r1) == 0 and len(r2) == 0:
        other_saves.append(d)


In [ ]:
import textwrap
for d in other_saves:
    print(json.dumps(d,indent=2))

{
  "spell": "Aura of Purity",
  "saving_throw": "on",
  "desc": " Purifying energy radiates from you in an aura with a 30-foot radius.\nUntil the spell ends, the aura moves with you, centered on you. Each\nnon-hostile creature in the aura (including you) can\u2019t become\ndiseased, has resistance to poison damage, and has advantage on saving\nthrows against effects that cause any of the following conditions:\nblinded, charmed, deafened, frightened, paralyzed, poisoned, and\nstunned."
}
{
  "spell": "Bane",
  "saving_throw": "Charisma",
  "desc": " Up to three creatures of your choice that you can see within range\nmust make Charisma saving throws. Whenever a target that fails this\nsaving throw makes an attack roll or a saving throw before the spell\nends, the target must roll a d4 and subtract the number rolled from\nthe attack roll or saving throw."
}
{
  "spell": "Beacon of Hope",
  "saving_throw": "Wisdom",
  "desc": " This spell bestows hope and vitality. Choose any number of cr

In [303]:
##getting impact on succeed saves
for d in succeed_saves:
    # print(json.dumps(d,indent=2))
    desc = d['desc']
    pat = re.compile(r'(?<=saving throw or).*?(?=\.)')
    res = re.findall(pat,desc)
    if len(res) > 0:
        print(res)
    else:
        print(desc)

[' take 1d6 acid damage']
[' be charmed by you for the spell’s duration']
[' become frightened', ' use its movement on each of its turns to enter the area or move within reach of the target']
[' be banished']
[' become cursed for the duration of the spell']
 Necromantic energy washes over a creature of your choice that you can see within range, draining moisture and vitality from it. The target must make a Constitution saving throw. The target takes 8d8 necrotic damage on a failed save, or half as much damage on a successful one. This spell has no effect on undead or constructs. If you target a plant creature or a magical plant, it makes the saving throw with disadvantage, and the spell deals maximum damage to it. If you target a nonmagical plant that isn’t a creature, such as a tree or shrub, it doesn’t make a saving throw; it simply withers and dies. 
[' be blinded until the spell ends']
[' be lifted by the pillar']
[' follow the command on its next turn']
 This spell assaults and tw

In [280]:
len(require_saves)

187

## Pulling Character Class Information



In [114]:
import pandas as pd, numpy as np, requests
from bs4 import BeautifulSoup
baseURL = 'https://dnd5e.wikidot.com'
import re
resp = requests.get(baseURL)
char_classes = [
    "Artificer","Barbarian",
    "Bard","Cleric","Druid",
    "Fighter","Monk","Paladin",
    "Ranger","Rogue","Sorcerer",
    "Warlock","Wizard"
]
soup = BS(resp.content)
class_pages = []
for cl in char_classes:
    elems = soup(text=re.compile(r'{}'.format(cl)))
    elems = [e.parent for e in elems]
    for e in elems:
        if e.name == 'a' and e.text==cl:
            # print(e.name)
            # print(e)
            class_pages.append((cl,baseURL+e.attrs['href']))

display(
    class_pages
)

class_pages.pop(1)
display(class_pages)
class_files = []
for pg in class_pages:
    resp = requests.get(pg[1])
    soup = BS(resp.content)
    with open('./data_for_parsing/classes/{}.html'.format(pg[0]),'w') as f:
        f.write(soup.prettify())
        f.close()
    class_files.append(
        {
            'class':pg[0],
            'file':'./data_for_parsing/classes/{}.html'.format(pg[0]),
        }
    )

[('Artificer', 'https://dnd5e.wikidot.com/artificer'),
 ('Artificer', 'https://dnd5e.wikidot.com/wizard:artificer-ua'),
 ('Barbarian', 'https://dnd5e.wikidot.com/barbarian'),
 ('Bard', 'https://dnd5e.wikidot.com/bard'),
 ('Cleric', 'https://dnd5e.wikidot.com/cleric'),
 ('Druid', 'https://dnd5e.wikidot.com/druid'),
 ('Fighter', 'https://dnd5e.wikidot.com/fighter'),
 ('Monk', 'https://dnd5e.wikidot.com/monk'),
 ('Paladin', 'https://dnd5e.wikidot.com/paladin'),
 ('Ranger', 'https://dnd5e.wikidot.com/ranger'),
 ('Rogue', 'https://dnd5e.wikidot.com/rogue'),
 ('Sorcerer', 'https://dnd5e.wikidot.com/sorcerer'),
 ('Warlock', 'https://dnd5e.wikidot.com/warlock'),
 ('Wizard', 'https://dnd5e.wikidot.com/wizard')]

[('Artificer', 'https://dnd5e.wikidot.com/artificer'),
 ('Barbarian', 'https://dnd5e.wikidot.com/barbarian'),
 ('Bard', 'https://dnd5e.wikidot.com/bard'),
 ('Cleric', 'https://dnd5e.wikidot.com/cleric'),
 ('Druid', 'https://dnd5e.wikidot.com/druid'),
 ('Fighter', 'https://dnd5e.wikidot.com/fighter'),
 ('Monk', 'https://dnd5e.wikidot.com/monk'),
 ('Paladin', 'https://dnd5e.wikidot.com/paladin'),
 ('Ranger', 'https://dnd5e.wikidot.com/ranger'),
 ('Rogue', 'https://dnd5e.wikidot.com/rogue'),
 ('Sorcerer', 'https://dnd5e.wikidot.com/sorcerer'),
 ('Warlock', 'https://dnd5e.wikidot.com/warlock'),
 ('Wizard', 'https://dnd5e.wikidot.com/wizard')]

In [115]:
## Extract and store class tables
char_class_tables = {}
for file in class_files:
    soup = BS(open(file['file'],'r').read())
    tables = soup(id=re.compile(r'page-content.*'))
    # char_class_tables.append(tables[0])
    for table in tables:
        dfs = pd.read_html(table.decode_contents())
        for i in range(len(dfs)):
            if i == 0:
                tab = dfs[i]
                cl = tab.columns[0][0].split(' ')[1]
                tab['class'] = cl
                char_class_tables[cl] = tab.copy()
                # char_class_tables.append(dfs[i])
                # display(tab)

In [116]:
tmpArt = char_class_tables['Artificer'].copy()
tmpArt.columns = [
    'level','prof_bonus','features','infusions_known','infused_items','cantrips_known','1','2','3','4','5','drop','class'
]
for i in range(2,6):
    tmpArt['{}'.format(i)] = tmpArt['{}'.format(i)].str.replace('-','0').astype(int)
tmpArt['spellslots'] = tmpArt.apply(
    lambda row:{
        1:row['1'],
        2:row['2'],
        3:row['3'],
        4:row['4'],
        5:row['5']
    },
    axis=1
)
tmpArt.drop(columns=['1','2','3','4','5','drop'],inplace=True,axis=1)
tmpArt['level'] = tmpArt['level'].str[:-2].astype(int)
tmpArt['infused_items']= np.select(
    [tmpArt['infused_items']=='-'],
    [0],
    tmpArt['infused_items']
)
tmpArt['infusions_known']= np.select(
    [tmpArt['infusions_known']=='-'],
    [0],
    tmpArt['infusions_known']
)
tmpArt['features'] = tmpArt['features'].str.split(',')
for i,row in tmpArt.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpArt.at[i,'features'] = tmp
    
tmpArt['key'] = tmpArt['class'] + '-'+tmpArt['level'].astype(str)
tmpArt['class_specific'] = tmpArt.apply(
    lambda row: {
        'infusions_known':int(row['infusions_known']),
        'infused_items':int(row['infused_items']),
        'cantrips_known':int(row['cantrips_known'])
    },
    axis=1
)

tmpArt

,level,prof_bonus,features,infusions_known,infused_items,cantrips_known,class,spellslots,key,class_specific
0,1,2,"[Magical Tinkering, Spellcasting]",0,0,2,Artificer,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0}",Artificer-1,"{'infusions_known': 0, 'infused_items': 0, 'ca..."
1,2,2,[Infuse Item],4,2,2,Artificer,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0}",Artificer-2,"{'infusions_known': 4, 'infused_items': 2, 'ca..."
2,3,2,"[Artificer Specialist, The Right Tool for the...",4,2,2,Artificer,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0}",Artificer-3,"{'infusions_known': 4, 'infused_items': 2, 'ca..."
3,4,2,[Ability Score Improvement],4,2,2,Artificer,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0}",Artificer-4,"{'infusions_known': 4, 'infused_items': 2, 'ca..."
4,5,3,[Artificer Specialist feature],4,2,2,Artificer,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0}",Artificer-5,"{'infusions_known': 4, 'infused_items': 2, 'ca..."
5,6,3,[Tool Expertise],6,3,2,Artificer,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0}",Artificer-6,"{'infusions_known': 6, 'infused_items': 3, 'ca..."
6,7,3,[Flash of Genius],6,3,2,Artificer,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0}",Artificer-7,"{'infusions_known': 6, 'infused_items': 3, 'ca..."
7,8,3,[Ability Score Improvement],6,3,2,Artificer,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0}",Artificer-8,"{'infusions_known': 6, 'infused_items': 3, 'ca..."
8,9,4,[Artificer Specialist feature],6,3,2,Artificer,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0}",Artificer-9,"{'infusions_known': 6, 'infused_items': 3, 'ca..."
9,10,4,[Magic Item Adept],8,4,3,Artificer,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0}",Artificer-10,"{'infusions_known': 8, 'infused_items': 4, 'ca..."


In [117]:
# class specific = [{key:[{key:value}]}]
tmpBarb = char_class_tables['Barbarian'].copy()
tmpBarb.columns = ['level','prof_bonus','features','rages','rage_dmg','class']

tmpBarb['lvl'] = tmpBarb['level'].str[:-2].astype(int)
tmpBarb.loc[tmpBarb['rages']=='Unlimited','rages'] = '999' #edge case - unlimited rage
tmpBarb['features'] = tmpBarb['features'].str.split(',')
tmpBarb['key'] = tmpBarb['class']+'-'+tmpBarb['lvl'].astype(str)
tmpBarb['class_specific'] = tmpBarb.apply(
    lambda row: {
        'rages':int(row['rages']),
        'rage_dmg':int(row['rage_dmg'])
    },axis=1
)
for i,row in tmpBarb.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpBarb.at[i,'features'] = tmp
tmpBarb['level'] = tmpBarb['level'].str[:-2].astype(int)
tmpBarb

,level,prof_bonus,features,rages,rage_dmg,class,lvl,key,class_specific
0,1,2,"[Rage, Unarmored Defense]",2,2,Barbarian,1,Barbarian-1,"{'rages': 2, 'rage_dmg': 2}"
1,2,2,"[Reckless Attack, Danger Sense]",2,2,Barbarian,2,Barbarian-2,"{'rages': 2, 'rage_dmg': 2}"
2,3,2,"[Primal Path, Primal Knowledge (Optional)]",3,2,Barbarian,3,Barbarian-3,"{'rages': 3, 'rage_dmg': 2}"
3,4,2,[Ability Score Improvement],3,2,Barbarian,4,Barbarian-4,"{'rages': 3, 'rage_dmg': 2}"
4,5,3,"[Extra Attack, Fast Movement]",3,2,Barbarian,5,Barbarian-5,"{'rages': 3, 'rage_dmg': 2}"
5,6,3,[Path feature],4,2,Barbarian,6,Barbarian-6,"{'rages': 4, 'rage_dmg': 2}"
6,7,3,"[Feral Instinct, Instinctive Pounce (Optional)]",4,2,Barbarian,7,Barbarian-7,"{'rages': 4, 'rage_dmg': 2}"
7,8,3,[Ability Score Improvement],4,2,Barbarian,8,Barbarian-8,"{'rages': 4, 'rage_dmg': 2}"
8,9,4,[Brutal Critical (1 die)],4,3,Barbarian,9,Barbarian-9,"{'rages': 4, 'rage_dmg': 3}"
9,10,4,"[Path feature, Primal Knowledge (Optional)]",4,3,Barbarian,10,Barbarian-10,"{'rages': 4, 'rage_dmg': 3}"


In [ ]:
# class specific = [{key:[{key:value}]}]
tmpBard = char_class_tables['Bard'].copy()
tmpBard.columns = ['level','prof_bonus','features','cantrips_known','spells_known','1','2','3','4','5','6','7','8','9','class']
tmpBard['features'] = tmpBard['features'].str.split(',')
for i in range(2,10):
    tmpBard[f'{i}'] = tmpBard[f'{i}'].str.replace('-','0').astype(int)
tmpBard['spellslots'] = tmpBard.apply(
    lambda row: {1:row['1'],2:row['2'],3:row['3'],4:row['4'],5:row['5'],6:row['6'],7:row['7'],8:row['8'],9:row['9']},
    axis=1
)
tmpBard['class_specific'] = tmpBard.apply(
    lambda row:{
        'cantrips_known':int(row['cantrips_known']),
        'spells_known':int(row['spells_known'])
    },
    axis=1
)
for i,row in tmpBard.iterrows():
    tmp = []
    if isinstance(row['features'],float):
        tmpBard.at[i,'features'] = []
    else:
        for j in row['features']:
            if '’' in j:
                tmp.append(re.sub('’','\'\'',j))
            else:
                tmp.append(j)
    tmpBard.at[i,'features'] = tmp
tmpBard.drop(columns=['1','2','3','4','5','6','7','8','9'],inplace=True)
tmpBard['level'] = tmpBard['level'].str[:-2].astype(int)
tmpBard

,level,prof_bonus,features,cantrips_known,spells_known,class,spellslots,class_specific
0,1,2,"[Spellcasting, Bardic Inspiration (d6)]",2,4,Bard,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'cantrips_known': 2, 'spells_known': 4}"
1,2,2,"[Jack of All Trades, Song of Rest (d6), Mag...",2,5,Bard,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'cantrips_known': 2, 'spells_known': 5}"
2,3,2,"[Bard College, Expertise]",2,6,Bard,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'cantrips_known': 2, 'spells_known': 6}"
3,4,2,"[Ability Score Improvement, Bardic Versatili...",3,7,Bard,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'cantrips_known': 3, 'spells_known': 7}"
4,5,3,"[Bardic Inspiration (d8), Font of Inspiration]",3,8,Bard,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'cantrips_known': 3, 'spells_known': 8}"
5,6,3,"[Countercharm, Bard College feature]",3,9,Bard,"{1: 4, 2: 3, 3: 3, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'cantrips_known': 3, 'spells_known': 9}"
6,7,3,[],3,10,Bard,"{1: 4, 2: 3, 3: 3, 4: 1, 5: 0, 6: 0, 7: 0, 8: ...","{'cantrips_known': 3, 'spells_known': 10}"
7,8,3,"[Ability Score Improvement, Bardic Versatili...",3,11,Bard,"{1: 4, 2: 3, 3: 3, 4: 2, 5: 0, 6: 0, 7: 0, 8: ...","{'cantrips_known': 3, 'spells_known': 11}"
8,9,4,[Song of Rest (d8)],3,12,Bard,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 1, 6: 0, 7: 0, 8: ...","{'cantrips_known': 3, 'spells_known': 12}"
9,10,4,"[Bardic Inspiration (d10), Expertise, Magica...",4,14,Bard,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 2, 6: 0, 7: 0, 8: ...","{'cantrips_known': 4, 'spells_known': 14}"


In [ ]:
# class specific = [{key:[{key:value}]}]
tempCler = char_class_tables['Cleric'].copy()
tempCler.columns = ['level','prof_bonus','features','cantrips_known','1','2','3','4','5','6','7','8','9','class']
for i in range(2,10):
    tempCler[f'{i}'] = tempCler[f'{i}'].str.replace('-','0').astype(int)
tempCler['features'] = tempCler['features'].str.split(',')
tempCler['spellslots'] = tempCler.apply(
    lambda row: {1:row['1'],2:row['2'],3:row['3'],4:row['4'],5:row['5'],6:row['6'],7:row['7'],8:row['8'],9:row['9']},
    axis=1
)
tempCler['level'] = tempCler['level'].str[:-2].astype(int)
tempCler.drop(columns=['1','2','3','4','5','6','7','8','9'],inplace=True)
tempCler['key'] = tempCler['class'] + '-'+ tempCler['level'].astype(str)

tempCler['class_specific'] = tempCler.apply(
    lambda row:
        {'cantrips_known':int(row['cantrips_known'])}
    ,
    axis=1
)
for i,row in tempCler.iterrows():
    tmp = []
    if isinstance(row['features'],float):
        tempCler.at[i,'features'] = []
    else:
        for j in row['features']:
            if '’' in j:
                tmp.append(re.sub('’','\'\'',j))
            else:
                tmp.append(j)
    tempCler.at[i,'features'] = tmp

tempCler


,level,prof_bonus,features,cantrips_known,class,spellslots,key,class_specific
0,1,2,"[Spellcasting, Divine Domain]",3,Cleric,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Cleric-1,{'cantrips_known': 3}
1,2,2,"[Channel Divinity (x1), Divine Domain feature...",3,Cleric,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Cleric-2,{'cantrips_known': 3}
2,3,2,[],3,Cleric,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Cleric-3,{'cantrips_known': 3}
3,4,2,"[Ability Score Improvement, Cantrip Versatil...",4,Cleric,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Cleric-4,{'cantrips_known': 4}
4,5,3,[Destroy Undead (CR 1/2)],4,Cleric,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Cleric-5,{'cantrips_known': 4}
5,6,3,"[Channel Divinity (x2), Divine Domain feature]",4,Cleric,"{1: 4, 2: 3, 3: 3, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Cleric-6,{'cantrips_known': 4}
6,7,3,[],4,Cleric,"{1: 4, 2: 3, 3: 3, 4: 1, 5: 0, 6: 0, 7: 0, 8: ...",Cleric-7,{'cantrips_known': 4}
7,8,3,"[Ability Score Improvement, Destroy Undead (C...",4,Cleric,"{1: 4, 2: 3, 3: 3, 4: 2, 5: 0, 6: 0, 7: 0, 8: ...",Cleric-8,{'cantrips_known': 4}
8,9,4,[],4,Cleric,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 1, 6: 0, 7: 0, 8: ...",Cleric-9,{'cantrips_known': 4}
9,10,4,[Divine Intervention],5,Cleric,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 2, 6: 0, 7: 0, 8: ...",Cleric-10,{'cantrips_known': 5}


In [122]:
tmpDruid = char_class_tables['Druid'].copy()
tmpDruid.drop(columns=tmpDruid.columns[-2],inplace=True)
tmpDruid.columns = ['level','prof_bonus','features','cantrips_known','1','2','3','4','5','6','7','8','9','class']
for i in range(2,10):
    tmpDruid[f'{i}'] = tmpDruid[f'{i}'].str.replace('-','0').astype(int)
tmpDruid['features'] = tmpDruid['features'].str.split(',')
tmpDruid['spellslots'] = tmpDruid.apply(
    lambda row: {1:row['1'],2:row['2'],3:row['3'],4:row['4'],5:row['5'],6:row['6'],7:row['7'],8:row['8'],9:row['9']},
    axis=1
)
tmpDruid['level'] = tmpDruid['level'].str[:-2].astype(int)
tmpDruid.drop(columns=['1','2','3','4','5','6','7','8','9'],inplace=True)
tmpDruid['key'] = tmpDruid['class'] + '-'+ tmpDruid['level'].astype(str)
tmpDruid['class_specific'] = tmpDruid.apply(
    lambda row:
        {'cantrips_known':int(row['cantrips_known'])}
    ,
    axis=1
)
for i,row in tmpDruid.iterrows():
    tmp = []
    if isinstance(row['features'],float):
        tmpBarb.at[i,'features'] = []
    else:
        for j in row['features']:
            if '’' in j:
                tmp.append(re.sub('’','\'\'',j))
            else:
                tmp.append(j)
    tmpDruid.at[i,'features'] = tmp
tmpDruid

,level,prof_bonus,features,cantrips_known,class,spellslots,key,class_specific
0,1,2,"[Druidic, Spellcasting]",2,Druid,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Druid-1,{'cantrips_known': 2}
1,2,2,"[Wild Shape, Druid Circle, Wild Companion (...",2,Druid,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Druid-2,{'cantrips_known': 2}
2,3,2,[],2,Druid,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Druid-3,{'cantrips_known': 2}
3,4,2,"[Wild Shape improvement, Ability Score Improv...",3,Druid,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Druid-4,{'cantrips_known': 3}
4,5,3,[],3,Druid,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Druid-5,{'cantrips_known': 3}
5,6,3,[Druid Circle feature],3,Druid,"{1: 4, 2: 3, 3: 3, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Druid-6,{'cantrips_known': 3}
6,7,3,[],3,Druid,"{1: 4, 2: 3, 3: 3, 4: 1, 5: 0, 6: 0, 7: 0, 8: ...",Druid-7,{'cantrips_known': 3}
7,8,3,"[Wild Shape improvement, Ability Score Improv...",3,Druid,"{1: 4, 2: 3, 3: 3, 4: 2, 5: 0, 6: 0, 7: 0, 8: ...",Druid-8,{'cantrips_known': 3}
8,9,4,[],3,Druid,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 1, 6: 0, 7: 0, 8: ...",Druid-9,{'cantrips_known': 3}
9,10,4,[Druid Circle feature],4,Druid,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 2, 6: 0, 7: 0, 8: ...",Druid-10,{'cantrips_known': 4}


In [ ]:
tmpFight = char_class_tables['Fighter'].copy()
tmpFight.columns = ['level','prof_bonus','features','?','??','class']
tmpFight.drop(columns=['?','??'],inplace=True)
tmpFight['features'] = tmpFight['features'].str.split(',')
tmpFight['class_specific'] = None
tmpFight['level'] = tmpFight['level'].str[:-2].astype(int)
for i,row in tempCler.iterrows():
    tmp = []
    if isinstance(row['features'],float):
        tmpBarb.at[i,'features'] = []
    else:
        for j in row['features']:
            if '’' in j:
                tmp.append(re.sub('’','\'\'',j))
            else:
                tmp.append(j)
    tempCler.at[i,'features'] = tmp
tmpFight

,level,prof_bonus,features,class,class_specific
0,1,2,"[Fighting Style, Second Wind]",Fighter,None
1,2,2,[Action Surge (x1)],Fighter,None
2,3,2,[Martial Archetype],Fighter,None
3,4,2,"[Ability Score Improvement, Martial Versatil...",Fighter,None
4,5,3,[Extra Attack (x1)],Fighter,None
5,6,3,"[Ability Score Improvement, Martial Versatil...",Fighter,None
6,7,3,[Martial Archetype feature],Fighter,None
7,8,3,"[Ability Score Improvement, Martial Versatil...",Fighter,None
8,9,4,[Indomitable (x1)],Fighter,None
9,10,4,[Martial Archetype feature],Fighter,None


In [ ]:
tmpMonk = char_class_tables['Monk'].copy()
tmpMonk.columns = ['level','prof_bonus','martial arts','ki_points','unarmored_mvmt','features','class']
tmpMonk['level'] = tmpMonk['level'].str[:-2].astype(int)
tmpMonk['features'] = tmpMonk['features'].str.split(',')
tmpMonk.loc[0,'unarmored_mvmt'] = '+0 ft.'
tmpMonk.loc[0,'ki_points'] = '0'
tmpMonk['ki_points'] = tmpMonk['ki_points'].astype(int)
tmpMonk['unarmored_mvmt'] = tmpMonk['unarmored_mvmt']#.str#.split(' ',expand=True)[1].astype(int)
tmpMonk['unarmored_mvmt'] = tmpMonk['unarmored_mvmt'].str.extract('(\d+)').astype(int)
tmpMonk[['ma_d','ma_s']] = tmpMonk['martial arts'].str.extract(r'(\d+)d(\d+)')
tmpMonk['martial_arts'] = tmpMonk.apply(
    lambda row: {int(row['ma_d']):int(row['ma_s'])},
axis=1
)
tmpMonk['class_specific'] = tmpMonk.apply(
    lambda row: {
        'ki_points':row['ki_points'],
        'unarmored_mvmt':row['unarmored_mvmt'],
        'martial_arts_dice':int(row['ma_s']),
        'martial_arts_dmg':int(row['ma_d']) #need a handler...
    },
    axis=1
)
for i,row in tmpMonk.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpMonk.at[i,'features'] = tmp
tmpMonk.drop(columns=['martial arts','ma_d','ma_s'],inplace=True)
tmpMonk

,level,prof_bonus,ki_points,unarmored_mvmt,features,class,martial_arts,class_specific
0,1,2,0,0,"[Unarmored Defense, Martial Arts]",Monk,{1: 4},"{'ki_points': 0, 'unarmored_mvmt': 0, 'martial..."
1,2,2,2,10,"[Ki, Unarmored Movement, Dedicated Weapon (...",Monk,{1: 4},"{'ki_points': 2, 'unarmored_mvmt': 10, 'martia..."
2,3,2,3,10,"[Monastic Tradition, Deflect Missiles, Ki-F...",Monk,{1: 4},"{'ki_points': 3, 'unarmored_mvmt': 10, 'martia..."
3,4,2,4,10,"[Ability Score Improvement, Slow Fall, Quic...",Monk,{1: 4},"{'ki_points': 4, 'unarmored_mvmt': 10, 'martia..."
4,5,3,5,10,"[Extra Attack, Stunning Strike, Focused Aim...",Monk,{1: 6},"{'ki_points': 5, 'unarmored_mvmt': 10, 'martia..."
5,6,3,6,15,"[Ki-Empowered Strikes, Monastic Tradition fea...",Monk,{1: 6},"{'ki_points': 6, 'unarmored_mvmt': 15, 'martia..."
6,7,3,7,15,"[Evasion, Stillness of Mind]",Monk,{1: 6},"{'ki_points': 7, 'unarmored_mvmt': 15, 'martia..."
7,8,3,8,15,[Ability Score Improvement],Monk,{1: 6},"{'ki_points': 8, 'unarmored_mvmt': 15, 'martia..."
8,9,4,9,15,[Unarmored Movement improvement],Monk,{1: 6},"{'ki_points': 9, 'unarmored_mvmt': 15, 'martia..."
9,10,4,10,20,[Purity of Body],Monk,{1: 6},"{'ki_points': 10, 'unarmored_mvmt': 20, 'marti..."


In [ ]:
tmpPal = char_class_tables['Paladin'].copy()
tmpPal.columns = ['level','prof_bonus','features','1','2','3','4','5','?','??','???','????','class']
tmpPal.drop(columns=['?','??','???','????'],inplace=True)
tmpPal['level'] = tmpPal['level'].str[:-2].astype(int)
tmpPal['features'] = tmpPal['features'].str.split(',')
for i in range(1,6):
    tmpPal[f'{i}'] = tmpPal[f'{i}'].str.replace('-','0').astype(int)
tmpPal['spellslots'] = tmpPal.apply(
    lambda row: {1:row['1'],2:row['2'],3:row['3'],4:row['4'],5:row['5']},
    axis=1
)
tmpPal.drop(columns=['1','2','3','4','5'],axis=1,inplace=True)
tmpPal['key'] = tmpPal['class'] + '-' + tmpPal['level'].astype(str)
tmpPal['class_specific'] = None
for i,row in tmpPal.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpPal.at[i,'features'] = tmp
tmpPal

,level,prof_bonus,features,class,spellslots,key,class_specific
0,1,2,"[Divine Sense, Lay on Hands]",Paladin,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 0}",Paladin-1,None
1,2,2,"[Fighting Style, Spellcasting, Divine Smite]",Paladin,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0}",Paladin-2,None
2,3,2,"[Divine Health, Sacred Oath, Harness Divine...",Paladin,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0}",Paladin-3,None
3,4,2,"[Ability Score Improvement, Martial Versatil...",Paladin,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0}",Paladin-4,None
4,5,3,[Extra Attack],Paladin,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0}",Paladin-5,None
5,6,3,[Aura of Protection],Paladin,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0}",Paladin-6,None
6,7,3,[Sacred Oath feature],Paladin,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0}",Paladin-7,None
7,8,3,"[Ability Score Improvement, Martial Versatil...",Paladin,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0}",Paladin-8,None
8,9,4,NaN,Paladin,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0}",Paladin-9,None
9,10,4,[Aura of Courage],Paladin,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0}",Paladin-10,None


In [ ]:
tmpRng = char_class_tables['Ranger'].copy()
tmpRng.columns = ['level','prof_bonus','features','spells_known','1','2','3','4','5','?','??','???','????','class']
tmpRng.drop(columns=['?',"??",'???','????'],inplace=True)
tmpRng['level'] = tmpRng['level'].str[:-2].astype(int)
tmpRng['key'] = tmpRng['class'] + '-' + tmpRng['level'].astype(str)
tmpRng['features'] = tmpRng['features'].str.split(',')
for i in range(1,6):
    tmpRng[f'{i}'] = tmpRng[f'{i}'].str.replace('-','0').astype(int)
tmpRng['spellslots'] = tmpRng.apply(
    lambda row: {1:row['1'],2:row['2'],3:row['3'],4:row['4'],5:row['5']},
    axis=1
)
tmpRng.drop(columns=['1','2','3','4','5'],axis=1,inplace=True)
tmpRng.loc[0,'spells_known'] = '0'
tmpRng['spells_known'] = tmpRng['spells_known'].astype(int)
tmpRng['class_specific'] = tmpRng.apply(
    lambda row: {
        'spells_known':row['spells_known']
    }, axis=1
)
for i,row in tmpRng.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpRng.at[i,'features'] = tmp
tmpRng

,level,prof_bonus,features,spells_known,class,key,spellslots,class_specific
0,1,2,"[Favored Enemy, Natural Explorer, Deft Expl...",0,Ranger,Ranger-1,"{1: 0, 2: 0, 3: 0, 4: 0, 5: 0}",{'spells_known': 0}
1,2,2,"[Fighting Style, Spellcasting, Spellcasting...",2,Ranger,Ranger-2,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0}",{'spells_known': 2}
2,3,2,"[Primeval Awareness, Ranger Conclave, Prima...",3,Ranger,Ranger-3,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0}",{'spells_known': 3}
3,4,2,"[Ability Score Improvement, Martial Versatil...",3,Ranger,Ranger-4,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0}",{'spells_known': 3}
4,5,3,[Extra Attack],4,Ranger,Ranger-5,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0}",{'spells_known': 4}
5,6,3,"[Favored Enemy Improvement, Natural Explorer ...",4,Ranger,Ranger-6,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0}",{'spells_known': 4}
6,7,3,[Ranger Conclave feature],5,Ranger,Ranger-7,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0}",{'spells_known': 5}
7,8,3,"[Ability Score Improvement, Land's Stride, ...",5,Ranger,Ranger-8,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0}",{'spells_known': 5}
8,9,4,NaN,6,Ranger,Ranger-9,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0}",{'spells_known': 6}
9,10,4,"[Natural Explorer Improvement, Hide in Plain ...",6,Ranger,Ranger-10,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0}",{'spells_known': 6}


In [ ]:
tmpRogue = char_class_tables['Rogue'].copy()
tmpRogue.columns = ['level','prof_bonus','sneak_attack','features','?','??','class']
tmpRogue.drop(columns=['?',"??"],inplace=True)
tmpRogue['level'] = tmpRogue['level'].str[:-2].astype(int)
tmpRogue['key'] = tmpRogue['class'] + '-' + tmpRogue['level'].astype(str)
tmpRogue['features'] = tmpRogue['features'].str.split(',')
tmpRogue[['sa_d','sa_s']] = tmpRogue['sneak_attack'].str.extract(r'(\d+)d(\d+)')
tmpRogue['sneak_attack'] = tmpRogue.apply(
    lambda row:{str(row['sa_d']):int(row['sa_s'])},
    axis=1
)
tmpRogue['class_specific'] = tmpRogue.apply(
    lambda row: {
        "sneak_attack_die":int(row['sa_d']),
        "sneak_attack_dmg":int(row['sa_s'])
    },
    axis=1
)
for i,row in tmpRogue.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpRogue.at[i,'features'] = tmp
tmpRogue.drop(columns=['sa_s','sa_d'],inplace=True)
tmpRogue

,level,prof_bonus,sneak_attack,features,class,key,class_specific
0,1,2,{'1': 6},"[Expertise, Sneak Attack, Thieves' Cant]",Rogue,Rogue-1,"{'sneak_attack_die': '1', 'sneak_attack_dmg': ..."
1,2,2,{'1': 6},[Cunning Action],Rogue,Rogue-2,"{'sneak_attack_die': '1', 'sneak_attack_dmg': ..."
2,3,2,{'2': 6},"[Roguish Archetype, Steady Aim (Optional)]",Rogue,Rogue-3,"{'sneak_attack_die': '2', 'sneak_attack_dmg': ..."
3,4,2,{'2': 6},[Ability Score Improvement],Rogue,Rogue-4,"{'sneak_attack_die': '2', 'sneak_attack_dmg': ..."
4,5,3,{'3': 6},[Uncanny Dodge],Rogue,Rogue-5,"{'sneak_attack_die': '3', 'sneak_attack_dmg': ..."
5,6,3,{'3': 6},[Expertise],Rogue,Rogue-6,"{'sneak_attack_die': '3', 'sneak_attack_dmg': ..."
6,7,3,{'4': 6},[Evasion],Rogue,Rogue-7,"{'sneak_attack_die': '4', 'sneak_attack_dmg': ..."
7,8,3,{'4': 6},[Ability Score Improvement],Rogue,Rogue-8,"{'sneak_attack_die': '4', 'sneak_attack_dmg': ..."
8,9,4,{'5': 6},[Roguish Archetype feature],Rogue,Rogue-9,"{'sneak_attack_die': '5', 'sneak_attack_dmg': ..."
9,10,4,{'5': 6},[Ability Score Improvement],Rogue,Rogue-10,"{'sneak_attack_die': '5', 'sneak_attack_dmg': ..."


In [ ]:
tmpSorc = char_class_tables['Sorcerer'].copy()
tmpSorc.columns = ['level','prof_bonus','sorc_points','features','cantrips_known','spells_known','1','2','3','4','5','6','7','8','9','class']
tmpSorc['level'] = tmpSorc['level'].str[:-2].astype(int)
tmpSorc['key'] = tmpSorc['class'] + '-' + tmpSorc['level'].astype(str)
tmpSorc['features'] = tmpSorc['features'].str.split(',')
for i in range(2,10):
    tmpSorc[f'{i}'] = tmpSorc[f'{i}'].str.replace('-','0').astype(int)
tmpSorc['spellslots'] = tmpSorc.apply(
    lambda row: {1:row['1'],2:row['2'],3:row['3'],4:row['4'],5:row['5'],6:row['6'],7:row['7'],8:row['8'],9:row['9']},
    axis=1
)
tmpSorc.loc[tmpSorc['sorc_points'].str.contains('-'),'sorc_points'] = 0
tmpSorc['class_specific'] = tmpSorc.apply(
    lambda row: {
        'sorc_points':int(row['sorc_points']),
        'cantrips_known':int(row['cantrips_known']),
        'spells_known':int(row['spells_known'])
    },axis=1
)
for i,row in tmpSorc.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpSorc.at[i,'features'] = tmp
tmpSorc.drop(columns=['1','2','3','4','5','6','7','8','9'],inplace=True)
tmpSorc

,level,prof_bonus,sorc_points,features,cantrips_known,spells_known,class,key,spellslots,class_specific
0,1,2,0,"[Spellcasting, Sorcerous Origin]",4,2,Sorcerer,Sorcerer-1,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'sorc_points': 0, 'cantrips_known': 4, 'spell..."
1,2,2,2,[Font of Magic],4,3,Sorcerer,Sorcerer-2,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'sorc_points': 2, 'cantrips_known': 4, 'spell..."
2,3,2,3,[Metamagic],4,4,Sorcerer,Sorcerer-3,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'sorc_points': 3, 'cantrips_known': 4, 'spell..."
3,4,2,4,"[Ability Score Improvement, Sorcerous Versat...",5,5,Sorcerer,Sorcerer-4,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'sorc_points': 4, 'cantrips_known': 5, 'spell..."
4,5,3,5,[Magical Guidance (Optional)],5,6,Sorcerer,Sorcerer-5,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'sorc_points': 5, 'cantrips_known': 5, 'spell..."
5,6,3,6,[Sorcerous Origin feature],5,7,Sorcerer,Sorcerer-6,"{1: 4, 2: 3, 3: 3, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...","{'sorc_points': 6, 'cantrips_known': 5, 'spell..."
6,7,3,7,NaN,5,8,Sorcerer,Sorcerer-7,"{1: 4, 2: 3, 3: 3, 4: 1, 5: 0, 6: 0, 7: 0, 8: ...","{'sorc_points': 7, 'cantrips_known': 5, 'spell..."
7,8,3,8,"[Ability Score Improvement, Sorcerous Versat...",5,9,Sorcerer,Sorcerer-8,"{1: 4, 2: 3, 3: 3, 4: 2, 5: 0, 6: 0, 7: 0, 8: ...","{'sorc_points': 8, 'cantrips_known': 5, 'spell..."
8,9,4,9,NaN,5,10,Sorcerer,Sorcerer-9,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 1, 6: 0, 7: 0, 8: ...","{'sorc_points': 9, 'cantrips_known': 5, 'spell..."
9,10,4,10,[Metamagic],6,11,Sorcerer,Sorcerer-10,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 2, 6: 0, 7: 0, 8: ...","{'sorc_points': 10, 'cantrips_known': 6, 'spel..."


In [ ]:
tmpWar = char_class_tables['Warlock'].copy()
tmpWar.columns = ['level','prof_bonus','features','cantrips_known','spells_known','spell_slots','slot_level','invocations_known','class']
tmpWar['level'] = tmpWar['level'].str[:-2].astype(int)
tmpWar['key'] = tmpWar['class'] + '-' + tmpWar['level'].astype(str)
tmpWar['features'] = tmpWar['features'].str.split(',')
tmpWar['slot_level'] = tmpWar['slot_level'].str[:-2].astype(int)
tmpWar.loc[0,'invocations_known'] = '0' 
tmpWar['invocations_known'] = tmpWar['invocations_known'].astype(int)
tmpWar['class_specific'] = tmpWar.apply(
    lambda row: {
        'cantrips_known':row['cantrips_known'],
        'spells_known':row['spells_known'],
        'invocations_known':row['invocations_known']
    },
    axis=1
)
tmpWar['spellslots'] = tmpWar.apply(
    lambda row: {int(row['slot_level']):int(row['spell_slots'])},
    axis=1
)
for i,row in tmpWar.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpWar.at[i,'features'] = tmp
tmpWar

,level,prof_bonus,features,cantrips_known,spells_known,spell_slots,slot_level,invocations_known,class,key,class_specific,spellslots
0,1,2,"[Otherworldly Patron, Pact Magic]",2,2,1,1,0,Warlock,Warlock-1,"{'cantrips_known': 2, 'spells_known': 2, 'invo...",{1: 1}
1,2,2,[Eldritch Invocations],2,3,2,1,2,Warlock,Warlock-2,"{'cantrips_known': 2, 'spells_known': 3, 'invo...",{1: 2}
2,3,2,[Pact Boon],2,4,2,2,2,Warlock,Warlock-3,"{'cantrips_known': 2, 'spells_known': 4, 'invo...",{2: 2}
3,4,2,"[Ability Score Improvement, Eldritch Versati...",3,5,2,2,2,Warlock,Warlock-4,"{'cantrips_known': 3, 'spells_known': 5, 'invo...",{2: 2}
4,5,3,NaN,3,6,2,3,3,Warlock,Warlock-5,"{'cantrips_known': 3, 'spells_known': 6, 'invo...",{3: 2}
5,6,3,[Otherworldly Patron feature],3,7,2,3,3,Warlock,Warlock-6,"{'cantrips_known': 3, 'spells_known': 7, 'invo...",{3: 2}
6,7,3,NaN,3,8,2,4,4,Warlock,Warlock-7,"{'cantrips_known': 3, 'spells_known': 8, 'invo...",{4: 2}
7,8,3,"[Ability Score Improvement, Eldritch Versati...",3,9,2,4,4,Warlock,Warlock-8,"{'cantrips_known': 3, 'spells_known': 9, 'invo...",{4: 2}
8,9,4,NaN,3,10,2,5,5,Warlock,Warlock-9,"{'cantrips_known': 3, 'spells_known': 10, 'inv...",{5: 2}
9,10,4,[Otherworldly Patron feature],4,10,2,5,5,Warlock,Warlock-10,"{'cantrips_known': 4, 'spells_known': 10, 'inv...",{5: 2}


In [ ]:
tmpWiz = char_class_tables['Wizard'].copy()
tmpWiz.columns = ['level','prof_bonus','features','cantrips_known','1','2','3','4','5','6','7','8','9','class']
for i in range(2,10):
    tmpWiz[f'{i}'] = tmpWiz[f'{i}'].str.replace('-','0').astype(int)
tmpWiz['spellslots'] = tmpWiz.apply(
    lambda row: {1:row['1'],2:row['2'],3:row['3'],4:row['4'],5:row['5'],6:row['6'],7:row['7'],8:row['8'],9:row['9']},
    axis=1
)
tmpWiz.drop(columns=['1','2','3','4','5','6','7','8','9'],inplace=True)
tmpWiz['level'] = tmpWiz['level'].str[:-2].astype(int)
tmpWiz['key'] = tmpWiz['class'] + '-' + tmpWiz['level'].astype(str)
tmpWiz['features'] = tmpWiz['features'].str.split(',')
tmpWiz['class_specific'] = tmpWiz.apply(
    lambda row: {'cantrips_known':row['cantrips_known']},
    axis=1
)
for i,row in tmpWiz.iterrows():
    tmp = []
    for j in row['features']:
        if '’' in j:
            tmp.append(re.sub('’','\'\'',j))
        else:
            tmp.append(j)
    tmpWiz.at[i,'features'] = tmp
tmpWiz


,level,prof_bonus,features,cantrips_known,class,spellslots,key,class_specific
0,1,2,"[Spellcasting, Arcane Recovery]",3,Wizard,"{1: 2, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Wizard-1,{'cantrips_known': 3}
1,2,2,[Arcane Tradition],3,Wizard,"{1: 3, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Wizard-2,{'cantrips_known': 3}
2,3,2,[Cantrip Formulas (Optional)],3,Wizard,"{1: 4, 2: 2, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Wizard-3,{'cantrips_known': 3}
3,4,2,[Ability Score Improvement],4,Wizard,"{1: 4, 2: 3, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Wizard-4,{'cantrips_known': 4}
4,5,3,NaN,4,Wizard,"{1: 4, 2: 3, 3: 2, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Wizard-5,{'cantrips_known': 4}
5,6,3,[Arcane Tradition feature],4,Wizard,"{1: 4, 2: 3, 3: 3, 4: 0, 5: 0, 6: 0, 7: 0, 8: ...",Wizard-6,{'cantrips_known': 4}
6,7,3,NaN,4,Wizard,"{1: 4, 2: 3, 3: 3, 4: 1, 5: 0, 6: 0, 7: 0, 8: ...",Wizard-7,{'cantrips_known': 4}
7,8,3,[Ability Score Improvement],4,Wizard,"{1: 4, 2: 3, 3: 3, 4: 2, 5: 0, 6: 0, 7: 0, 8: ...",Wizard-8,{'cantrips_known': 4}
8,9,4,NaN,4,Wizard,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 1, 6: 0, 7: 0, 8: ...",Wizard-9,{'cantrips_known': 4}
9,10,4,[Arcane Tradition feature],5,Wizard,"{1: 4, 2: 3, 3: 3, 4: 3, 5: 2, 6: 0, 7: 0, 8: ...",Wizard-10,{'cantrips_known': 5}


In [117]:
frames = []
common = ['level','prof_bonus','features','class','spellslots','class_specific']
for cl in [tmpBarb,tmpBard,tmpDruid,tmpFight,tmpMonk,tmpPal,tmpRng,tmpRogue,tempCler,tmpSorc,tmpWar,tmpWiz,tmpArt]:
    drop = set(cl.columns).difference(common)
    frames.append(cl.drop(columns=drop,axis=1))
pd.concat(frames)

,level,prof_bonus,features,class,class_specific,spellslots
0,1,2,"[Rage, Unarmored Defense]",Barbarian,"{'rages': 2, 'rage_dmg': 2}",NaN
1,2,2,"[Reckless Attack, Danger Sense]",Barbarian,"{'rages': 2, 'rage_dmg': 2}",NaN
2,3,2,"[Primal Path, Primal Knowledge (Optional)]",Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
3,4,2,[Ability Score Improvement],Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
4,5,3,"[Extra Attack, Fast Movement]",Barbarian,"{'rages': 3, 'rage_dmg': 2}",NaN
...,...,...,...,...,...,...
15,16,5,[Ability Score Improvement],Artificer,"{'infusions_known': 10, 'infused_items': 5, 'c...","{1: 4, 2: 3, 3: 3, 4: 2, 5: 0}"
16,17,6,[-],Artificer,"{'infusions_known': 10, 'infused_items': 5, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 1}"
17,18,6,[Magic Item Master],Artificer,"{'infusions_known': 12, 'infused_items': 6, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 1}"
18,19,6,[Ability Score Improvement],Artificer,"{'infusions_known': 12, 'infused_items': 6, 'c...","{1: 4, 2: 3, 3: 3, 4: 3, 5: 2}"


In [ ]:
class_tables_clean = pd.concat(frames)


In [ ]:
class_tables_clean.to_csv('./data/class_by_level.csv',index=False)

Other class table information:
* hit dice
* hit points
* hit points per level
* proficiencies: (separate table)
* starting equipment
* pathway (e.g. schools/traditions, paths, pacts, etc)


In [111]:
# for file in class_files:
#     f = open(file['file'],'r').read()
#     soup = BS(f)
soup = BS(open(class_files[0]['file'],'r').read())

# elems = soup(text=re.compile(r'{}'.format(cl)))
soup('strong',text=re.compile(r'Hit Dice:'))

def get_hp_data(soup):
    hp_data = [e.parent for e in soup('strong',text=re.compile(r'Hit Dice:'))]
    hp_text = hp_data[0].decode_contents()
    hp_text = re.sub('\n','',hp_text)
    hp_text = hp_text.split('<br/><strong>')
    res = {}
    for i in hp_text:
        t = i.split('</strong>')
        for j in range(len(t)):
            t[j] = re.sub('<strong>','',t[j])
            t[j] = t[j].strip()
        res[t[0]] = t[1]
    hp = {}
    hp['hit_dice']=re.findall(r'(\d+)d(\d+)',res['Hit Dice:'])
    hp['hit_dice']={int(hp['hit_dice'][0][0]):int(hp['hit_dice'][0][1])}
    hp['start_hp']=re.findall(r'(\d+)',res['Hit Points at 1st Level:'])[0]+'+CON'
    hp['lvl_hp']=re.findall(r'(\d+)d(\d+)',res['Hit Points at Higher Levels:'])
    hp['lvl_hp']={int(hp['lvl_hp'][0][0]):int(hp['lvl_hp'][0][1])}
    hp['alt_lvl_hp'] = re.findall(r'\(or (\d+)\)',res['Hit Points at Higher Levels:'])[0]+'+CON'
    # print(hp)
    return hp
get_hp_data(soup)

NameError: name 'class_files' is not defined

In [ ]:
def get_starting_equipment(soup):
    equip_data = [e.next.next.next for e in soup('p',text=re.compile('You start with.*?(?=:)'))][0]
    
    equip_data = [e.text.strip() for e in equip_data('li')]
    for i in range(len(equip_data)):
        if '’' in equip_data[i]:
            equip_data[i] = re.sub('’','\'\'')
get_starting_equipment(soup)

['any two simple weapons',
 'a light crossbow and 20 bolts',
 '(a) studded leather armor or (b) scale mail',
 'thieves’ tools and a dungeoneer’s pack']

In [ ]:
def get_proficiencies(soup):
    prof_data = [e.parent for e in soup('strong',text=re.compile(r'Armor:'))][0]
    prof_data = prof_data.decode_contents()
    prof_data = re.sub('\n','',prof_data)
    prof_data = prof_data.split('<br/><strong>')
    dat = {}
    for rec in prof_data:
        t = rec.split('</strong>')
        k,v = t[0],t[1]
        k = re.sub('<strong>','',k).strip()
        v = [q.strip().lower() for q in v.split(',')]
        for i in range(len(v)):
            if '’' in v[i]:
                v[i] = re.sub('’','\'\'',v[i])
        dat[k] = v
    dat['Skills:'].pop(0)
    return dat
get_proficiencies(soup)

{'Armor:': ['light armor', 'medium armor', 'shields'],
 'Weapons:': ['simple weapons'],
 'Tools:': ['thieves’ tools',
  'tinker’s tools',
  'one type of artisan’s tools of your choice'],
 'Saving Throws:': ['constitution', 'intelligence'],
 'Skills:': ['history',
  'investigation',
  'medicine',
  'nature',
  'perception',
  'sleight of hand']}

In [443]:
class_tables_clean.loc[class_tables_clean['class']=='Artificer']

,level,prof_bonus,features,rages,rage_dmg,class,lvl,key,cantrips_known,spells_known,...,sorc_points,spell_slots,slot_level,invocations_known,infusions_known,infused_items,char_level,hp,proficiencies,start_equip
0,1st,2,"[Magical Tinkering, Spellcasting]",NaN,NaN,Artificer,NaN,Artificer-1,2.0,NaN,...,NaN,NaN,NaN,NaN,0,0,1,NaN,NaN,
1,2nd,2,[Infuse Item],NaN,NaN,Artificer,NaN,Artificer-2,2.0,NaN,...,NaN,NaN,NaN,NaN,4,2,2,NaN,NaN,
2,3rd,2,"[Artificer Specialist, The Right Tool for the...",NaN,NaN,Artificer,NaN,Artificer-3,2.0,NaN,...,NaN,NaN,NaN,NaN,4,2,3,NaN,NaN,
3,4th,2,[Ability Score Improvement],NaN,NaN,Artificer,NaN,Artificer-4,2.0,NaN,...,NaN,NaN,NaN,NaN,4,2,4,NaN,NaN,
4,5th,3,[Artificer Specialist feature],NaN,NaN,Artificer,NaN,Artificer-5,2.0,NaN,...,NaN,NaN,NaN,NaN,4,2,5,NaN,NaN,
5,6th,3,[Tool Expertise],NaN,NaN,Artificer,NaN,Artificer-6,2.0,NaN,...,NaN,NaN,NaN,NaN,6,3,6,NaN,NaN,
6,7th,3,[Flash of Genius],NaN,NaN,Artificer,NaN,Artificer-7,2.0,NaN,...,NaN,NaN,NaN,NaN,6,3,7,NaN,NaN,
7,8th,3,[Ability Score Improvement],NaN,NaN,Artificer,NaN,Artificer-8,2.0,NaN,...,NaN,NaN,NaN,NaN,6,3,8,NaN,NaN,
8,9th,4,[Artificer Specialist feature],NaN,NaN,Artificer,NaN,Artificer-9,2.0,NaN,...,NaN,NaN,NaN,NaN,6,3,9,NaN,NaN,
9,10th,4,[Magic Item Adept],NaN,NaN,Artificer,NaN,Artificer-10,3.0,NaN,...,NaN,NaN,NaN,NaN,8,4,10,NaN,NaN,


In [478]:
class_tables_clean.to_csv('./data/class_by_level.csv',index=False)

In [475]:
# class_tables_clean[['hp','proficiencies','start_equip']] = '','',''
class_start_data = pd.DataFrame({
    'class':[],
    'hp':[],
    'equip':[],
    'proficiencies':[]
})
class_start_data['equip'] = class_start_data['equip'].astype('object')
for file in class_files:
    f = open(file['file'],'r').read()
    soup = BS(f)
    # print(file['class'])
    # class_start_data.loc[len(class_start_data)] = {'class':file['class'],'hp':'','equip':'','proficiencies':''}
    try:
        profs = get_proficiencies(soup)
        # class_start_data.loc[class_start_data.loc[len(class_start_data)-1]['class']==file['class'],'proficiencies'] = profs
        # print('\tproficiencies: {}'.format(get_proficiencies(soup)))
    except:
        profs = None
        print('error getting proficiencies for {}'.format(file['class']))
    try:
        hp = get_hp_data(soup)
        # class_tables_clean.loc[class_tables_clean['class']==file['class'],'hp'] = hp
        # print('\thp:{}'.format(get_hp_data(soup)))
    except:
        hp = None
        print('error getting hp for {}'.format(file['class']))
    try:
        equip = get_starting_equipment(soup)
        # print('\tequipment: {}'.format(get_starting_equipment(soup)))
        # class_tables_clean.loc[class_tables_clean['class']==file['class'],'start_equip'] = equip
    except:
        equip = None
        print('\terror getting equipment for {}'.format(file['class']))
    if equip is None:
        equip = []
    d = {'class':file['class'],'hp':hp,'equip':equip,'proficiencies':profs}
    # print(d)
    class_start_data.loc[len(class_start_data)] = {'class':file['class'],'hp':hp,'equip':equip,'proficiencies':profs}
    # class_start_data.loc[len(class_start_data)] = d
class_start_data

	error getting equipment for Druid


,class,hp,equip,proficiencies
0,Artificer,"{'hit_dice': {1: 8}, 'start_hp': '8+CON', 'lvl...","[any two simple weapons, a light crossbow and ...","{'Armor:': ['light armor', 'medium armor', 'sh..."
1,Barbarian,"{'hit_dice': {1: 12}, 'start_hp': '12+CON', 'l...",[(a) a greataxe or (b) any martial melee weapo...,"{'Armor:': ['light armor', 'medium armor', 'sh..."
2,Bard,"{'hit_dice': {1: 8}, 'start_hp': '8+CON', 'lvl...","[(a) a rapier, (b) a longsword, or (c) any sim...","{'Armor:': ['light armor'], 'Weapons:': ['simp..."
3,Cleric,"{'hit_dice': {1: 8}, 'start_hp': '8+CON', 'lvl...",[(a) a mace or (b) a warhammer (if proficient)...,"{'Armor:': ['light armor', 'medium armor', 'sh..."
4,Druid,"{'hit_dice': {1: 8}, 'start_hp': '8+CON', 'lvl...",[],"{'Armor:': ['light armor', 'medium armor', 'sh..."
5,Fighter,"{'hit_dice': {1: 10}, 'start_hp': '10+CON', 'l...","[(a) chain mail or (b) leather, longbow, and 2...","{'Armor:': ['all armor', 'shields'], 'Weapons:..."
6,Monk,"{'hit_dice': {1: 8}, 'start_hp': '8+CON', 'lvl...","[(a) a shortsword or (b) any simple weapon, (a...","{'Armor:': ['none'], 'Weapons:': ['simple weap..."
7,Paladin,"{'hit_dice': {1: 10}, 'start_hp': '10+CON', 'l...",[(a) a martial weapon and a shield or (b) two ...,"{'Armor:': ['all armor', 'shields'], 'Weapons:..."
8,Ranger,"{'hit_dice': {1: 10}, 'start_hp': '10+CON', 'l...","[(a) scale mail or (b) leather armor, (a) two ...","{'Armor:': ['light armor', 'medium armor', 'sh..."
9,Rogue,"{'hit_dice': {1: 8}, 'start_hp': '8+CON', 'lvl...","[(a) a rapier or (b) a shortsword, (a) a short...","{'Armor:': ['light armor'], 'Weapons:': ['simp..."


In [477]:
class_start_data.to_csv('./data/class_start_data.csv',index=False)

In [397]:
## other class table information
common = set()
all_paras = []
for file in class_files:
    soup = BS(open(file['file'],'r').read())
    elems = soup(id=re.compile(r'toc\d+'))
    # print(file['class'])
    para_list = set()
    for e in elems:
        # print("\t{}".format(e.text.strip()))
        para_list.add(e.text.strip())
    all_paras.append({'class':file['class'],'paras':para_list})
# print(all_paras)
for i in range(len(all_paras)):
    if i == 0:
        common = all_paras[i]['paras']
    else:
        common = common.intersection(all_paras[i]['paras'])

common

# char_class_tables = []
# for file in class_files:
#     soup = BS(open(file['file'],'r').read())
#     tables = soup(id=re.compile(r'page-content.*'))
#     for table in tables:
#         dfs = pd.read_html(table.decode_contents())
#         for df in dfs:
#             display(df)

{'Ability Score Improvement',
 'Class Features',
 'Equipment',
 'Hit Points',
 'Proficiencies'}

## Pulling Weapon Information

In [54]:
import requests, pandas as pd
from bs4 import BeautifulSoup as BS
url = 'https://dnd5e.wikidot.com/weapons'
resp = requests.get(url)
soup = BS(resp.content)

tables = []

weapons_tables = soup(id='page-content')
print(len(weapons_tables))
print(weapons_tables)
for w in weapons_tables:
    t = pd.read_html(w.decode_contents())
    tables.append(t)
# type(weapons_tables[0].decode_contents())

1
[<div id="page-content">
<div class="floatright" id="toc">
<div id="toc-action-bar"><a href="javascript:;" onclick="WIKIDOT.page.listeners.foldToc(event)">Fold</a><a href="javascript:;" onclick="WIKIDOT.page.listeners.unfoldToc(event)" style="display: none">Unfold</a></div>
<div class="title">Table of Contents</div>
<div id="toc-list">
<div style="margin-left: 1em;"><a href="#toc0">Simple Weapons</a></div>
<div style="margin-left: 1em;"><a href="#toc1">Martial Weapons</a></div>
<div style="margin-left: 1em;"><a href="#toc2">Ammunition</a></div>
<div style="margin-left: 1em;"><a href="#toc3">Setting Specific Weapons</a></div>
<div style="margin-left: 2em;"><a href="#toc4">Dragonlance</a></div>
<div style="margin-left: 2em;"><a href="#toc5">Ebberon</a></div>
<div style="margin-left: 1em;"><a href="#toc6">Weapon Properties</a></div>
<div style="margin-left: 2em;"><a href="#toc7">Improvised Weapons</a></div>
<div style="margin-left: 2em;"><a href="#toc8">Silvered Weapons</a></div>
<div s

In [95]:
for table in tables:
    for t in table:
        if isinstance(t,pd.DataFrame):
            # print(list(t.columns))
            display(t)

Simple Melee Weapons                                                \
                     Name                   Cost                 Damage   
0                    Club                   1 sp        1d4 bludgeoning   
1                  Dagger                   2 gp           1d4 piercing   
2               Greatclub                   2 sp        1d8 bludgeoning   
3                 Handaxe                   5 gp           1d6 slashing   
4                 Javelin                   5 sp           1d6 piercing   
5            Light hammer                   2 gp        1d4 bludgeoning   
6                    Mace                   5 gp        1d6 bludgeoning   
7            Quarterstaff                   2 sp        1d6 bludgeoning   
8                  Sickle                   1 gp           1d4 slashing   
9                   Spear                   1 gp           1d6 piercing   
10  Simple Ranged Weapons  Simple Ranged Weapons  Simple Ranged Weapons   
11                   Name                   Cost                 Damage   
12        Crossbow, light                  25 gp           1d8 piercing   
13                   Dart                   5 cp           1d4 piercing   
14               Shortbow                  25 gp           1d6 piercing   
15                  Sling                   1 sp        1d4 bludgeoning   

                                                                            
                   Weight                                       Properties  
0                   2 lb.                                            Light  
1                   1 lb.                   Finesse, light, thrown (20/60)  
2                  10 lb.                                       Two-handed  
3                   2 lb.                            Light, thrown (20/60)  
4                   2 lb.                                  Thrown (30/120)  
5                   2 lb.                            Light, thrown (20/60)  
6                   4 lb.                                                —  
7                   4 lb.                                  Versatile (1d8)  
8                   2 lb.                                            Light  
9                   3 lb.                  Thrown (20/60), versatile (1d8)  
10  Simple Ranged Weapons                            Simple Ranged Weapons  
11                 Weight                                       Properties  
12                  5 lb.  Ammunition, range (80/320), loading, two-handed  
13                1/4 lb.                          Finesse, thrown (20/60)  
14                  2 lb.           Ammunition, range (80/320), two-handed  
15                      —                       Ammunition, range (30/120)

Martial Melee Weapons                                                  \
                      Name                    Cost                  Damage   
0                Battleaxe                   10 gp            1d8 slashing   
1                    Flail                   10 gp         1d8 bludgeoning   
2                   Glaive                   20 gp           1d10 slashing   
3                 Greataxe                   30 gp           1d12 slashing   
4               Greatsword                   50 gp            2d6 slashing   
5                  Halberd                   20 gp           1d10 slashing   
6                    Lance                   10 gp           1d12 piercing   
7                Longsword                   15 gp            1d8 slashing   
8                     Maul                   10 gp         2d6 bludgeoning   
9              Morningstar                   15 gp            1d8 piercing   
10                    Pike                    5 gp           1d10 piercing   
11                  Rapier                   25 gp            1d8 piercing   
12                Scimitar                   25 gp            1d6 slashing   
13              Shortsword                   10 gp            1d6 piercing   
14                 Trident                    5 gp            1d6 piercing   
15                War pick                    5 gp            1d8 piercing   
16               Warhammer                   15 gp         1d8 bludgeoning   
17                    Whip                    2 gp            1d4 slashing   
18  Martial Ranged Weapons  Martial Ranged Weapons  Martial Ranged Weapons   
19                    Name                    Cost                  Damage   
20                 Blowgun                   10 gp              1 piercing   
21          Crossbow, hand                   75 gp            1d6 piercing   
22         Crossbow, heavy                   50 gp           1d10 piercing   
23                 Longbow                   50 gp            1d8 piercing   
24                     Net                    1 gp                       —   

                                                                               
                    Weight                                         Properties  
0                    4 lb.                                   Versatile (1d10)  
1                    2 lb.                                                  —  
2                    6 lb.                           Heavy, reach, two-handed  
3                    7 lb.                                  Heavy, two-handed  
4                    6 lb.                                  Heavy, two-handed  
5                    6 lb.                           Heavy, reach, two-handed  
6                    6 lb.                                     Reach, special  
7                    3 lb.                                   Versatile (1d10)  
8                   10 lb.                                  Heavy, two-handed  
9                    4 lb.                                                  —  
10                  18 lb.                           Heavy, reach, two-handed  
11                   2 lb.                                            Finesse  
12                   3 lb.                                     Finesse, light  
13                   2 lb.                                     Finesse, light  
14                   4 lb.                    Thrown (20/60), versatile (1d8)  
15                   2 lb.                                                  —  
16                   2 lb.                                   Versatile (1d10)  
17                   3 lb.                                     Finesse, reach  
18  Martial Ranged Weapons                             Martial Ranged Weapons  
19                  Weight                                         Properties  
20                   1 lb.                Ammunition, range (25/100), loading  
21                   3 lb.         Ammunition, range (30/1

,Ammunition,Cost,Weight
0,Arrows (20),1 gp,1 lb.
1,Blowgun needles (50),1 gp,1 lb.
2,Crossbow bolts (20),1 gp,1½ lb.
3,Sling bullets (20),4 cp,1½ lb.


Martial Melee Weapons        \
                                                Name  Cost   
0  Hoopak * A hoopak is a sturdy stick with a sli...  1 gp   

                                                            \
                                             Damage Weight   
0  1d6 piercing (melee) or 1d4 bludgeoning (ranged)  2 lb.   

                                                      
                                          Properties  
0  Ammunition (range 40/160), finesse, special, t...

Martial Melee Weapons                                                  
                     Name    Cost        Damage Weight           Properties
0  Double-Bladed Scimitar  100 gp  2d4 Slashing  6 lb.  Special, two-handed

,Property,Description
0,Ammunition,You can use a weapon that has the ammunition p...
1,Finesse,"When making an attack with a finesse weapon, y..."
2,Heavy,Creatures that are Small or Tiny have disadvan...
3,Light,"A light weapon is small and easy to handle, ma..."
4,Loading,Because of the time required to load this weap...
5,Range,A weapon that can be used to make a ranged att...
6,Reach,This weapon adds 5 feet to your reach when you...
7,Special,A weapon with the special property has unusual...
8,Thrown,"If a weapon has the thrown property, you can t..."
9,Two-Handed,This weapon requires two hands to use. This pr...


In [96]:
res = []
for table in tables:
    print(len(table))
    for i in range(2):
        print(i)
        t = table[i].copy()
        if isinstance(t,pd.DataFrame):
            # print(list(t.columns))
            typ = list(t.columns)[0][0]
            t.columns = ['Name','Cost','Damage','Weight','Properties']
            split = t.loc[t['Name'].str.contains('Weapons')].index[0]
            display("split_index = {}".format(split))
            t2 = t.loc[split:].copy()
            t = t.loc[:split-1]
            t2['type'] = t2.iloc[0]['Name']
            # display(t2)
            # t2.reset_index(inplace=True)
            # t2.drop(columns=['index'],axis=1,inplace=True)
            # t2 = t2.loc[3:]
            t['type'] = typ
            display(t)
            t2.reset_index(inplace=True)
            t2.drop(columns='index',inplace=True)
            t2 = t2.loc[2:]
            display(t2)
            res.append(t)
            res.append(t2)

6
0


'split_index = 10'

,Name,Cost,Damage,Weight,Properties,type
0,Club,1 sp,1d4 bludgeoning,2 lb.,Light,Simple Melee Weapons
1,Dagger,2 gp,1d4 piercing,1 lb.,"Finesse, light, thrown (20/60)",Simple Melee Weapons
2,Greatclub,2 sp,1d8 bludgeoning,10 lb.,Two-handed,Simple Melee Weapons
3,Handaxe,5 gp,1d6 slashing,2 lb.,"Light, thrown (20/60)",Simple Melee Weapons
4,Javelin,5 sp,1d6 piercing,2 lb.,Thrown (30/120),Simple Melee Weapons
5,Light hammer,2 gp,1d4 bludgeoning,2 lb.,"Light, thrown (20/60)",Simple Melee Weapons
6,Mace,5 gp,1d6 bludgeoning,4 lb.,—,Simple Melee Weapons
7,Quarterstaff,2 sp,1d6 bludgeoning,4 lb.,Versatile (1d8),Simple Melee Weapons
8,Sickle,1 gp,1d4 slashing,2 lb.,Light,Simple Melee Weapons
9,Spear,1 gp,1d6 piercing,3 lb.,"Thrown (20/60), versatile (1d8)",Simple Melee Weapons


,Name,Cost,Damage,Weight,Properties,type
2,"Crossbow, light",25 gp,1d8 piercing,5 lb.,"Ammunition, range (80/320), loading, two-handed",Simple Ranged Weapons
3,Dart,5 cp,1d4 piercing,1/4 lb.,"Finesse, thrown (20/60)",Simple Ranged Weapons
4,Shortbow,25 gp,1d6 piercing,2 lb.,"Ammunition, range (80/320), two-handed",Simple Ranged Weapons
5,Sling,1 sp,1d4 bludgeoning,—,"Ammunition, range (30/120)",Simple Ranged Weapons


1


'split_index = 18'

,Name,Cost,Damage,Weight,Properties,type
0,Battleaxe,10 gp,1d8 slashing,4 lb.,Versatile (1d10),Martial Melee Weapons
1,Flail,10 gp,1d8 bludgeoning,2 lb.,—,Martial Melee Weapons
2,Glaive,20 gp,1d10 slashing,6 lb.,"Heavy, reach, two-handed",Martial Melee Weapons
3,Greataxe,30 gp,1d12 slashing,7 lb.,"Heavy, two-handed",Martial Melee Weapons
4,Greatsword,50 gp,2d6 slashing,6 lb.,"Heavy, two-handed",Martial Melee Weapons
5,Halberd,20 gp,1d10 slashing,6 lb.,"Heavy, reach, two-handed",Martial Melee Weapons
6,Lance,10 gp,1d12 piercing,6 lb.,"Reach, special",Martial Melee Weapons
7,Longsword,15 gp,1d8 slashing,3 lb.,Versatile (1d10),Martial Melee Weapons
8,Maul,10 gp,2d6 bludgeoning,10 lb.,"Heavy, two-handed",Martial Melee Weapons
9,Morningstar,15 gp,1d8 piercing,4 lb.,—,Martial Melee Weapons


,Name,Cost,Damage,Weight,Properties,type
2,Blowgun,10 gp,1 piercing,1 lb.,"Ammunition, range (25/100), loading",Martial Ranged Weapons
3,"Crossbow, hand",75 gp,1d6 piercing,3 lb.,"Ammunition, range (30/120), light, loading",Martial Ranged Weapons
4,"Crossbow, heavy",50 gp,1d10 piercing,18 lb.,"Ammunition, range (100/400), heavy, loading, t...",Martial Ranged Weapons
5,Longbow,50 gp,1d8 piercing,2 lb.,"Ammunition, range (150/600), heavy, two-Handed",Martial Ranged Weapons
6,Net,1 gp,—,3 lb.,"Special, thrown (5/15)",Martial Ranged Weapons


Weapons will include

Weapons, Firearms, Explosives

Armor

In [97]:
weps = pd.concat(res)
weps.reset_index(inplace=True)
weps.drop(columns=['index'],axis=1,inplace=True)

weps.loc[weps['Damage'].str.contains('—'),'Damage'] = None
# weps.loc[weps['dmg_die'].str.contains('—'),'dmg_die'] = None
# weps.at[weps['dmg_die'].str.contains('—'),'dmg'] = {}
weps.loc[weps['Properties'].str.contains('—'),'Properties'] = None
weps.loc[weps['Weight'].str.contains('—'),'Weight'] = '0 lb.'

weps[['cost','cost_unit']] = weps['Cost'].str.split(' ',expand=True)
# weps.drop(columns='Cost',inplace=True)
weps['type'] = weps['type'].str.split(' ',expand=True)[1]
weps[['dmg_die','dmg_type']] =weps['Damage'].str.split(' ',expand=True)
# weps.drop(columns='Damage',inplace=True)
weps[['dmg_die_qty','dmg_die_size']] = weps['dmg_die'].str.split('d',expand=True)
# weps.drop(columns='dmg_die',inplace=True)

weps['dmg'] = None
weps['dmg'] = weps.apply(
    lambda row: {row['dmg_type']:{int(row['dmg_die_qty']):int(row['dmg_die_size'])}} if row['dmg_die_size'] is not None else {},
    axis=1
)
weps

,Name,Cost,Damage,Weight,Properties,type,cost,cost_unit,dmg_die,dmg_type,dmg_die_qty,dmg_die_size,dmg
0,Club,1 sp,1d4 bludgeoning,2 lb.,Light,Melee,1,sp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}}
1,Dagger,2 gp,1d4 piercing,1 lb.,"Finesse, light, thrown (20/60)",Melee,2,gp,1d4,piercing,1,4,{'piercing': {1: 4}}
2,Greatclub,2 sp,1d8 bludgeoning,10 lb.,Two-handed,Melee,2,sp,1d8,bludgeoning,1,8,{'bludgeoning': {1: 8}}
3,Handaxe,5 gp,1d6 slashing,2 lb.,"Light, thrown (20/60)",Melee,5,gp,1d6,slashing,1,6,{'slashing': {1: 6}}
4,Javelin,5 sp,1d6 piercing,2 lb.,Thrown (30/120),Melee,5,sp,1d6,piercing,1,6,{'piercing': {1: 6}}
5,Light hammer,2 gp,1d4 bludgeoning,2 lb.,"Light, thrown (20/60)",Melee,2,gp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}}
6,Mace,5 gp,1d6 bludgeoning,4 lb.,None,Melee,5,gp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}}
7,Quarterstaff,2 sp,1d6 bludgeoning,4 lb.,Versatile (1d8),Melee,2,sp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}}
8,Sickle,1 gp,1d4 slashing,2 lb.,Light,Melee,1,gp,1d4,slashing,1,4,{'slashing': {1: 4}}
9,Spear,1 gp,1d6 piercing,3 lb.,"Thrown (20/60), versatile (1d8)",Melee,1,gp,1d6,piercing,1,6,{'piercing': {1: 6}}


In [98]:
weps['props'] = weps['Properties'].str.split(', ')
for i, row in weps.iterrows():
    if row['props'] is None:
        weps.at[i,'props'] = []
    if row['dmg_die'] is None:
        weps.at[i,'dmg'] = {}
# weps.at[weps['props'].isna(),'props'] = []
weps[['normal_range','long_range']] = None,None
weps

,Name,Cost,Damage,Weight,Properties,type,cost,cost_unit,dmg_die,dmg_type,dmg_die_qty,dmg_die_size,dmg,props,normal_range,long_range
0,Club,1 sp,1d4 bludgeoning,2 lb.,Light,Melee,1,sp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},[Light],None,None
1,Dagger,2 gp,1d4 piercing,1 lb.,"Finesse, light, thrown (20/60)",Melee,2,gp,1d4,piercing,1,4,{'piercing': {1: 4}},"[Finesse, light, thrown (20/60)]",None,None
2,Greatclub,2 sp,1d8 bludgeoning,10 lb.,Two-handed,Melee,2,sp,1d8,bludgeoning,1,8,{'bludgeoning': {1: 8}},[Two-handed],None,None
3,Handaxe,5 gp,1d6 slashing,2 lb.,"Light, thrown (20/60)",Melee,5,gp,1d6,slashing,1,6,{'slashing': {1: 6}},"[Light, thrown (20/60)]",None,None
4,Javelin,5 sp,1d6 piercing,2 lb.,Thrown (30/120),Melee,5,sp,1d6,piercing,1,6,{'piercing': {1: 6}},[Thrown (30/120)],None,None
5,Light hammer,2 gp,1d4 bludgeoning,2 lb.,"Light, thrown (20/60)",Melee,2,gp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},"[Light, thrown (20/60)]",None,None
6,Mace,5 gp,1d6 bludgeoning,4 lb.,None,Melee,5,gp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[],None,None
7,Quarterstaff,2 sp,1d6 bludgeoning,4 lb.,Versatile (1d8),Melee,2,sp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[Versatile (1d8)],None,None
8,Sickle,1 gp,1d4 slashing,2 lb.,Light,Melee,1,gp,1d4,slashing,1,4,{'slashing': {1: 4}},[Light],None,None
9,Spear,1 gp,1d6 piercing,3 lb.,"Thrown (20/60), versatile (1d8)",Melee,1,gp,1d6,piercing,1,6,{'piercing': {1: 6}},"[Thrown (20/60), versatile (1d8)]",None,None


In [99]:
rng = weps['Properties'].str.extract(r'(?<=range \()(\d+)/(\d+)(?=\))')
thrown = weps['Properties'].str.extract(r'(?<=thrown \()(\d+)/(\d+)(?=\))')
reach = weps.loc[~(weps['Properties'].isna())&(weps['Properties'].str.contains('reach|Reach'))][['normal_range','long_range']].copy()
reach.columns = [0,1]
reach[[0,1]] = 10,10
reach
ranges = pd.concat([rng.dropna(),thrown.dropna(),reach]).sort_index()
ranges.columns = ['normal_range','long_range']
ranges

weps[['normal_range','long_range']] = ranges
weps

,Name,Cost,Damage,Weight,Properties,type,cost,cost_unit,dmg_die,dmg_type,dmg_die_qty,dmg_die_size,dmg,props,normal_range,long_range
0,Club,1 sp,1d4 bludgeoning,2 lb.,Light,Melee,1,sp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},[Light],NaN,NaN
1,Dagger,2 gp,1d4 piercing,1 lb.,"Finesse, light, thrown (20/60)",Melee,2,gp,1d4,piercing,1,4,{'piercing': {1: 4}},"[Finesse, light, thrown (20/60)]",20,60
2,Greatclub,2 sp,1d8 bludgeoning,10 lb.,Two-handed,Melee,2,sp,1d8,bludgeoning,1,8,{'bludgeoning': {1: 8}},[Two-handed],NaN,NaN
3,Handaxe,5 gp,1d6 slashing,2 lb.,"Light, thrown (20/60)",Melee,5,gp,1d6,slashing,1,6,{'slashing': {1: 6}},"[Light, thrown (20/60)]",20,60
4,Javelin,5 sp,1d6 piercing,2 lb.,Thrown (30/120),Melee,5,sp,1d6,piercing,1,6,{'piercing': {1: 6}},[Thrown (30/120)],NaN,NaN
5,Light hammer,2 gp,1d4 bludgeoning,2 lb.,"Light, thrown (20/60)",Melee,2,gp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},"[Light, thrown (20/60)]",20,60
6,Mace,5 gp,1d6 bludgeoning,4 lb.,None,Melee,5,gp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[],NaN,NaN
7,Quarterstaff,2 sp,1d6 bludgeoning,4 lb.,Versatile (1d8),Melee,2,sp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[Versatile (1d8)],NaN,NaN
8,Sickle,1 gp,1d4 slashing,2 lb.,Light,Melee,1,gp,1d4,slashing,1,4,{'slashing': {1: 4}},[Light],NaN,NaN
9,Spear,1 gp,1d6 piercing,3 lb.,"Thrown (20/60), versatile (1d8)",Melee,1,gp,1d6,piercing,1,6,{'piercing': {1: 6}},"[Thrown (20/60), versatile (1d8)]",NaN,NaN


In [100]:
weps['normal_range'].fillna(5,inplace=True)
weps['long_range'].fillna(5,inplace=True)
weps

,Name,Cost,Damage,Weight,Properties,type,cost,cost_unit,dmg_die,dmg_type,dmg_die_qty,dmg_die_size,dmg,props,normal_range,long_range
0,Club,1 sp,1d4 bludgeoning,2 lb.,Light,Melee,1,sp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},[Light],5,5
1,Dagger,2 gp,1d4 piercing,1 lb.,"Finesse, light, thrown (20/60)",Melee,2,gp,1d4,piercing,1,4,{'piercing': {1: 4}},"[Finesse, light, thrown (20/60)]",20,60
2,Greatclub,2 sp,1d8 bludgeoning,10 lb.,Two-handed,Melee,2,sp,1d8,bludgeoning,1,8,{'bludgeoning': {1: 8}},[Two-handed],5,5
3,Handaxe,5 gp,1d6 slashing,2 lb.,"Light, thrown (20/60)",Melee,5,gp,1d6,slashing,1,6,{'slashing': {1: 6}},"[Light, thrown (20/60)]",20,60
4,Javelin,5 sp,1d6 piercing,2 lb.,Thrown (30/120),Melee,5,sp,1d6,piercing,1,6,{'piercing': {1: 6}},[Thrown (30/120)],5,5
5,Light hammer,2 gp,1d4 bludgeoning,2 lb.,"Light, thrown (20/60)",Melee,2,gp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},"[Light, thrown (20/60)]",20,60
6,Mace,5 gp,1d6 bludgeoning,4 lb.,None,Melee,5,gp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[],5,5
7,Quarterstaff,2 sp,1d6 bludgeoning,4 lb.,Versatile (1d8),Melee,2,sp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[Versatile (1d8)],5,5
8,Sickle,1 gp,1d4 slashing,2 lb.,Light,Melee,1,gp,1d4,slashing,1,4,{'slashing': {1: 4}},[Light],5,5
9,Spear,1 gp,1d6 piercing,3 lb.,"Thrown (20/60), versatile (1d8)",Melee,1,gp,1d6,piercing,1,6,{'piercing': {1: 6}},"[Thrown (20/60), versatile (1d8)]",5,5


In [101]:
weps['mod'] = weps.apply(
    lambda row: ['STR'] if not 'Finesse' in row['props'] else ['STR','DEX'], axis=1
)

weps[['2hnd_die_qty','2hnd_die_size']] = weps['Properties'].str.extract(r'(?<=Versatile |versatile )\((\d+)d(\d+)\)')
weps['2hnd_die_qty'].fillna(0,inplace=True)
weps['2hnd_die_size'].fillna(0,inplace=True)
##keep, or delete?
# weps.loc[weps['2hnd_die_qty'].isna(),'2hnd_die_qty'] = weps.loc[weps['2hnd_die_qty'].isna(),'dmg_die_qty']
# weps.loc[weps['2hnd_die_size'].isna(),'2hnd_die_size'] = weps.loc[weps['2hnd_die_size'].isna(),'dmg_die_size']

# weps.loc[weps['2hnd_die_qty'].isna(),'2hnd_die_size'] = weps.loc[weps['2hnd_die_qty'].isna(),'dmg_die_size']

weps

,Name,Cost,Damage,Weight,Properties,type,cost,cost_unit,dmg_die,dmg_type,dmg_die_qty,dmg_die_size,dmg,props,normal_range,long_range,mod,2hnd_die_qty,2hnd_die_size
0,Club,1 sp,1d4 bludgeoning,2 lb.,Light,Melee,1,sp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},[Light],5,5,[STR],0,0
1,Dagger,2 gp,1d4 piercing,1 lb.,"Finesse, light, thrown (20/60)",Melee,2,gp,1d4,piercing,1,4,{'piercing': {1: 4}},"[Finesse, light, thrown (20/60)]",20,60,"[STR, DEX]",0,0
2,Greatclub,2 sp,1d8 bludgeoning,10 lb.,Two-handed,Melee,2,sp,1d8,bludgeoning,1,8,{'bludgeoning': {1: 8}},[Two-handed],5,5,[STR],0,0
3,Handaxe,5 gp,1d6 slashing,2 lb.,"Light, thrown (20/60)",Melee,5,gp,1d6,slashing,1,6,{'slashing': {1: 6}},"[Light, thrown (20/60)]",20,60,[STR],0,0
4,Javelin,5 sp,1d6 piercing,2 lb.,Thrown (30/120),Melee,5,sp,1d6,piercing,1,6,{'piercing': {1: 6}},[Thrown (30/120)],5,5,[STR],0,0
5,Light hammer,2 gp,1d4 bludgeoning,2 lb.,"Light, thrown (20/60)",Melee,2,gp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},"[Light, thrown (20/60)]",20,60,[STR],0,0
6,Mace,5 gp,1d6 bludgeoning,4 lb.,None,Melee,5,gp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[],5,5,[STR],0,0
7,Quarterstaff,2 sp,1d6 bludgeoning,4 lb.,Versatile (1d8),Melee,2,sp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[Versatile (1d8)],5,5,[STR],1,8
8,Sickle,1 gp,1d4 slashing,2 lb.,Light,Melee,1,gp,1d4,slashing,1,4,{'slashing': {1: 4}},[Light],5,5,[STR],0,0
9,Spear,1 gp,1d6 piercing,3 lb.,"Thrown (20/60), versatile (1d8)",Melee,1,gp,1d6,piercing,1,6,{'piercing': {1: 6}},"[Thrown (20/60), versatile (1d8)]",5,5,[STR],1,8


In [102]:
weps['2hnd_dmg'] = None
weps['2hnd_dmg'] = weps.apply(
    lambda row:
        {row['dmg_type']:{int(row['2hnd_die_qty']):int(row['2hnd_die_size'])}} if int(row['2hnd_die_qty']) > 0 else None
    ,
    axis=1
)
weps
weps.drop(columns=['Properties','Damage','dmg_die','dmg_type','dmg_die_qty','dmg_die_size'])#,	'2hnd_die_qty','2hnd_die_size'])

,Name,Cost,Weight,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,2hnd_die_qty,2hnd_die_size,2hnd_dmg
0,Club,1 sp,2 lb.,Melee,1,sp,{'bludgeoning': {1: 4}},[Light],5,5,[STR],0,0,None
1,Dagger,2 gp,1 lb.,Melee,2,gp,{'piercing': {1: 4}},"[Finesse, light, thrown (20/60)]",20,60,"[STR, DEX]",0,0,None
2,Greatclub,2 sp,10 lb.,Melee,2,sp,{'bludgeoning': {1: 8}},[Two-handed],5,5,[STR],0,0,None
3,Handaxe,5 gp,2 lb.,Melee,5,gp,{'slashing': {1: 6}},"[Light, thrown (20/60)]",20,60,[STR],0,0,None
4,Javelin,5 sp,2 lb.,Melee,5,sp,{'piercing': {1: 6}},[Thrown (30/120)],5,5,[STR],0,0,None
5,Light hammer,2 gp,2 lb.,Melee,2,gp,{'bludgeoning': {1: 4}},"[Light, thrown (20/60)]",20,60,[STR],0,0,None
6,Mace,5 gp,4 lb.,Melee,5,gp,{'bludgeoning': {1: 6}},[],5,5,[STR],0,0,None
7,Quarterstaff,2 sp,4 lb.,Melee,2,sp,{'bludgeoning': {1: 6}},[Versatile (1d8)],5,5,[STR],1,8,{'bludgeoning': {1: 8}}
8,Sickle,1 gp,2 lb.,Melee,1,gp,{'slashing': {1: 4}},[Light],5,5,[STR],0,0,None
9,Spear,1 gp,3 lb.,Melee,1,gp,{'piercing': {1: 6}},"[Thrown (20/60), versatile (1d8)]",5,5,[STR],1,8,{'piercing': {1: 8}}


In [103]:
weps['props'] = weps.apply(lambda row: [] if '—' in row['props'] else row['props'],axis=1)
weps

,Name,Cost,Damage,Weight,Properties,type,cost,cost_unit,dmg_die,dmg_type,dmg_die_qty,dmg_die_size,dmg,props,normal_range,long_range,mod,2hnd_die_qty,2hnd_die_size,2hnd_dmg
0,Club,1 sp,1d4 bludgeoning,2 lb.,Light,Melee,1,sp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},[Light],5,5,[STR],0,0,None
1,Dagger,2 gp,1d4 piercing,1 lb.,"Finesse, light, thrown (20/60)",Melee,2,gp,1d4,piercing,1,4,{'piercing': {1: 4}},"[Finesse, light, thrown (20/60)]",20,60,"[STR, DEX]",0,0,None
2,Greatclub,2 sp,1d8 bludgeoning,10 lb.,Two-handed,Melee,2,sp,1d8,bludgeoning,1,8,{'bludgeoning': {1: 8}},[Two-handed],5,5,[STR],0,0,None
3,Handaxe,5 gp,1d6 slashing,2 lb.,"Light, thrown (20/60)",Melee,5,gp,1d6,slashing,1,6,{'slashing': {1: 6}},"[Light, thrown (20/60)]",20,60,[STR],0,0,None
4,Javelin,5 sp,1d6 piercing,2 lb.,Thrown (30/120),Melee,5,sp,1d6,piercing,1,6,{'piercing': {1: 6}},[Thrown (30/120)],5,5,[STR],0,0,None
5,Light hammer,2 gp,1d4 bludgeoning,2 lb.,"Light, thrown (20/60)",Melee,2,gp,1d4,bludgeoning,1,4,{'bludgeoning': {1: 4}},"[Light, thrown (20/60)]",20,60,[STR],0,0,None
6,Mace,5 gp,1d6 bludgeoning,4 lb.,None,Melee,5,gp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[],5,5,[STR],0,0,None
7,Quarterstaff,2 sp,1d6 bludgeoning,4 lb.,Versatile (1d8),Melee,2,sp,1d6,bludgeoning,1,6,{'bludgeoning': {1: 6}},[Versatile (1d8)],5,5,[STR],1,8,{'bludgeoning': {1: 8}}
8,Sickle,1 gp,1d4 slashing,2 lb.,Light,Melee,1,gp,1d4,slashing,1,4,{'slashing': {1: 4}},[Light],5,5,[STR],0,0,None
9,Spear,1 gp,1d6 piercing,3 lb.,"Thrown (20/60), versatile (1d8)",Melee,1,gp,1d6,piercing,1,6,{'piercing': {1: 6}},"[Thrown (20/60), versatile (1d8)]",5,5,[STR],1,8,{'piercing': {1: 8}}


In [104]:
weps['Weight'].str[:-4]
list(weps['Weight'].str[:-4])
weps['wgt'] = list(weps['Weight'].str[:-4])

weps.drop(columns=['Damage','Properties','Cost','dmg_type','dmg_die_qty','dmg_die_size','2hnd_die_qty','2hnd_die_size'])

,Name,Weight,type,cost,cost_unit,dmg_die,dmg,props,normal_range,long_range,mod,2hnd_dmg,wgt
0,Club,2 lb.,Melee,1,sp,1d4,{'bludgeoning': {1: 4}},[Light],5,5,[STR],None,2
1,Dagger,1 lb.,Melee,2,gp,1d4,{'piercing': {1: 4}},"[Finesse, light, thrown (20/60)]",20,60,"[STR, DEX]",None,1
2,Greatclub,10 lb.,Melee,2,sp,1d8,{'bludgeoning': {1: 8}},[Two-handed],5,5,[STR],None,10
3,Handaxe,2 lb.,Melee,5,gp,1d6,{'slashing': {1: 6}},"[Light, thrown (20/60)]",20,60,[STR],None,2
4,Javelin,2 lb.,Melee,5,sp,1d6,{'piercing': {1: 6}},[Thrown (30/120)],5,5,[STR],None,2
5,Light hammer,2 lb.,Melee,2,gp,1d4,{'bludgeoning': {1: 4}},"[Light, thrown (20/60)]",20,60,[STR],None,2
6,Mace,4 lb.,Melee,5,gp,1d6,{'bludgeoning': {1: 6}},[],5,5,[STR],None,4
7,Quarterstaff,4 lb.,Melee,2,sp,1d6,{'bludgeoning': {1: 6}},[Versatile (1d8)],5,5,[STR],{'bludgeoning': {1: 8}},4
8,Sickle,2 lb.,Melee,1,gp,1d4,{'slashing': {1: 4}},[Light],5,5,[STR],None,2
9,Spear,3 lb.,Melee,1,gp,1d6,{'piercing': {1: 6}},"[Thrown (20/60), versatile (1d8)]",5,5,[STR],{'piercing': {1: 8}},3


In [105]:
weps['wgt'] = weps.apply(lambda row: eval("float({})".format(row['wgt'])),axis=1)


In [106]:
weps.drop(columns=['Weight','Damage','Properties','Cost','dmg_type','dmg_die_qty','dmg_die_size','2hnd_die_qty','2hnd_die_size','dmg_die'],inplace=True)

In [107]:
for i, row in weps.iterrows():
    if row['2hnd_dmg'] is None:
        weps.at[i,'dmg'] = {'1hnd':row['dmg']}
    else:
        weps.at[i,'dmg'] = {'1hnd':row['dmg'],'2hnd':row['2hnd_dmg']}
weps.drop(columns='2hnd_dmg',axis=1,inplace=True)

In [108]:
weps

,Name,type,cost,cost_unit,dmg,props,normal_range,long_range,mod,wgt
0,Club,Melee,1,sp,{'1hnd': {'bludgeoning': {1: 4}}},[Light],5,5,[STR],2.00
1,Dagger,Melee,2,gp,{'1hnd': {'piercing': {1: 4}}},"[Finesse, light, thrown (20/60)]",20,60,"[STR, DEX]",1.00
2,Greatclub,Melee,2,sp,{'1hnd': {'bludgeoning': {1: 8}}},[Two-handed],5,5,[STR],10.00
3,Handaxe,Melee,5,gp,{'1hnd': {'slashing': {1: 6}}},"[Light, thrown (20/60)]",20,60,[STR],2.00
4,Javelin,Melee,5,sp,{'1hnd': {'piercing': {1: 6}}},[Thrown (30/120)],5,5,[STR],2.00
5,Light hammer,Melee,2,gp,{'1hnd': {'bludgeoning': {1: 4}}},"[Light, thrown (20/60)]",20,60,[STR],2.00
6,Mace,Melee,5,gp,{'1hnd': {'bludgeoning': {1: 6}}},[],5,5,[STR],4.00
7,Quarterstaff,Melee,2,sp,"{'1hnd': {'bludgeoning': {1: 6}}, '2hnd': {'bl...",[Versatile (1d8)],5,5,[STR],4.00
8,Sickle,Melee,1,gp,{'1hnd': {'slashing': {1: 4}}},[Light],5,5,[STR],2.00
9,Spear,Melee,1,gp,"{'1hnd': {'piercing': {1: 6}}, '2hnd': {'pierc...","[Thrown (20/60), versatile (1d8)]",5,5,[STR],3.00


In [110]:
weps.to_csv('./data/weapons.csv',index=False)